# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Открытие файла с данными и изучение общей информации

In [1]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter


df = pd.read_csv('/datasets/data.csv') #открываем файл csv и сохраняем в датасет data

df.info() #выводим общую информацию и изучаем данные

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

В предоставленном файле, мы видем, что кол-во строк в файле 21525. Однако не все столбцы заполнены в полном объеме(в колонках *days_employed* и *total_income* заполнено только 19351 строк), судя по всему есть пропущенные/незаполненые значения. 

Названия столбцов написаны согласно правилам Pep8.

Окрытие первых 5 строк данных, сразу показывает нам, что не все данные в столбцах записаны в единном стиле (например, в столбце *education* данные записаны, полностью малыми, так и полностью большими буквами, а также с большой буквы), так и на первый взгляд с ошибочными данными (например, в столбце *days_employed* в строке 4 данные сильно отклоняются от других 340266.072047, что это? 932 года человек работал при возрасте 53 года?).

Непонятны записи в колонке *days_employed*, почему данные записаны в формате float (с плавающей точкой), если данные должны выражаться в днях, а в данных, есть цифры после запятой. Так же мы видим, что основные данные в данной колонке со знаком минус, что также не есть хорошо. Однако, вполне вероятно, что эти данные нам вовсе и не понадобятся для анализа.

Есть вопросы к сотрудникам, кто подготовил эти данные. Однако в данный момент у нас нет возможности задать их. Следовательно, будем работать с теми данными, что есть, по ходу определим, что и какие данные нам нужны для достижения цели, поставленной задачи.

Исходя из задания, нам нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. 

Если действовать строго по задаче и условиям, то нам нужно проанализировать строго только влияние семейного положения и кол-ва детей. Однако стоит отметить, что другие данные, такие как пол человека, образование, тип занятости и уровень дохода, также напрямую могут повлиять на итоги по критериям семейное положение и количества детей. 

## Шаг 2. Предобработка данных

В данном блоке мы проведем предобработку представленных нам данных.

Стоит отметить, что не только то, что мы указали в предыдущем шаге вызывает сомнения.
    
Сомнительно, что данные в колонке *total_income*, а именно после запятой, слишком много цифр.

Будем исправлять и корректировать по ходу предобработки данных.

### Обработка пропусков

Выясним в каких столбцах у нас есть пропущенные значения.

In [2]:
df.isna().sum() #с помощью метода isna() определим, где у нас есть пропущенные значения

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Пропуски обнаружены в одном столбце *total_income* кол-во пропусков 2174. Это существенное кол-во и удалять строки будет неправильно, т.к. они могут значительно повлиять на результаты. Поэтому их нужно заполнить. Данный столбец относится к количественным данным.

In [3]:
#определим какими данными заполнены пропуски в столбце *total_income*, 
#путем сортировки по этому столбцу и выводу последних 5 строк
df.sort_values(by='total_income').tail() 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости
21510,2,NaN,28,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,приобретение автомобиля


Видим, что данные пропуски заполнены значениями NaN. Не видно особых категорийных привязок к данным пропускам, поэтому вполне вероятно, что это могло произойти из-за сбоев при выгрузки, либо неверного заполнения данных. В связи с этим нужно уведомить коллег об этом.

Заполнять пустые значения будем медианными значениями с использованием группировки по типу занятости. Т.к. не правильно будет выставлять средние по всем данным (т.к. разброс может быть в группах велик) и также не правильно будет указать 0 (т.к. это может значительно исказить значения в итоговых группировках). 

Для этого выведим уникальные значения типа занятости и расчитаем, медианные значение доходов по этим группам. Если они будут значительно отличаться, значит наша гипотеза будет верной, и этими значениями мы заполним пропуски.

In [4]:
#определим все уникальные значения данных типа занятости 
#и сохраним их в list_income_type для дальнейшего использования
list_income_type = list(df['income_type'].unique())

In [5]:
#проверим действительно ли медианные значения в группах по типу занятости значительно отличаются
for i in list_income_type:
    print(f'Медианное значение по группе "{i}" - {round(df[df["income_type"] == i]["total_income"].median())}')

Медианное значение по группе "сотрудник" - 142594
Медианное значение по группе "пенсионер" - 118514
Медианное значение по группе "компаньон" - 172358
Медианное значение по группе "госслужащий" - 150448
Медианное значение по группе "безработный" - 131340
Медианное значение по группе "предприниматель" - 499163
Медианное значение по группе "студент" - 98202
Медианное значение по группе "в декрете" - 53829


Из данных видно, что медианные значения в разных группах значительно отличаются друг от друга. Соответственно, наша гипотеза верна.

Применим полученные значения к пропускам

In [6]:
#путем перебора значений подготовленных групп заменим значения на нужные нам
for i in list_income_type:
    df.loc[df['income_type'] == i, 'total_income'] = \
    df.loc[df['income_type'] == i, 'total_income'] \
    .fillna(df[df['income_type'] == i]['total_income'].median())

In [7]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [8]:
df['days_employed'] = df['days_employed'].abs()

Будем считать, что человек в среднем работает с 20 лет. Кто-то раньше начинает, кто-то позже.
Исходя из этого, вычислим рабочие дни так: возьмем возраст заемщика, вычтим 20 лет и умножим на кол-во рабочих дней в году (248 дней).

In [9]:
df.loc[df['days_employed'].isna(), 'days_employed'] = (df.loc[df['days_employed'].isna(), 'dob_years'] - 20) * 248

In [10]:
#Проверим на наличие пустот после заполнения значениями
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Все прошло успешно. Пустых значений в нашем рабочем dataframe более нет.

**Вывод**

Наша гипотеза подтвердилась, о том, что нам необходимо заменить пустые значения медианными значениями по группам. Мы успешно заполнили все пустоты.

### Замена типа данных

В исследуемой таблице, в столбце *total_income* (доход в месяц) значения указана в float формате, значений после точки слишком много, они как не могу повлиять на наши расчеты. Поэтому приведем значение к формату int. Это создаст еще и положительные эффекты, в виде простоты оперирования данными, а также более низкой нагрузки на сервер.

Для изменения типа данных, применим метод astype()

In [11]:
#методом astype() заменим значения колонки total_income на тип int
df['total_income'] = df['total_income'].astype('int') 
#методом astype() заменим значения колонки days_employed на тип int
df['days_employed'] = df['days_employed'].astype('int') 

In [12]:
df.head() #выведим первые 5 строк для проверки применения измениния типа

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


In [13]:
df.info() #проверим, действительно ли значения обновились на типовом уровне

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

Мы провели успешную замену значений в столбце *total_income* (доход в месяц) с типа float на тип данных int.

### Обработка дубликатов

Изначально мы должны проверить если в нашем dataframe дубликаты. Это мы можем сделать с помощью метода duplicated()

In [14]:
df.duplicated().sum() #проверим наличие дубликатов в dataframe

54

После проверки дубликатов стало 671. 

Так как ряд заполненной информации в столбцах записана не согласно инструкциям (а мы это уже видели при первичном изучении данных), ошибочно, приведем значения тех столбцов, где размещена текстовая информация к единому регистру.

Создадим list из столбцов со строковыми данными, чтобы перебрать его и перевести весь текст к нижнему регистру. Это позволит нам выявить дополнительные дубликаты с учетом ошибочного(не по инструкции) написания/заполнения текстовой информации и перезаписать значения.

In [15]:
#сформировали список столбцов
list_columns = ['gender', 'family_status', 'income_type', 'education', 'purpose']

#проходим по каждому столбцу из списка и приводим его текст к нижнему регистру
for column in list_columns:
    df[column] = df[column].str.lower()

Проверим сколько стало дубликатов после приведения в порядок данных.

In [16]:
df.duplicated().sum() #проверим дубликаты после приведения к нижнему регистру

71

Видим, что действительно после приведения к нижнему регистру, кол-во дублей увеличилось почти на 100 шт.

Далее, удалим все дубликаты с помощью метода drop_duplicates().

In [17]:
df = df.drop_duplicates().reset_index(drop=True) #Удаляем с использованием метода drop_duplicates() 
#и используем метод reset_index(drop=True) для того, чтобы не создавать столбец 
#со страрыми значениями индексов

Проведем проверку после удаления дубликатов.

In [18]:
df.duplicated().sum() #проверим наличие дубликатов, после применения метода удаления

0

In [19]:
df.info() #выведим общую информацию по данным

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21454 non-null  int64 
 1   days_employed     21454 non-null  int64 
 2   dob_years         21454 non-null  int64 
 3   education         21454 non-null  object
 4   education_id      21454 non-null  int64 
 5   family_status     21454 non-null  object
 6   family_status_id  21454 non-null  int64 
 7   gender            21454 non-null  object
 8   income_type       21454 non-null  object
 9   debt              21454 non-null  int64 
 10  total_income      21454 non-null  int64 
 11  purpose           21454 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


Возраст и трудовой стаж, в дальнейшем не будем использовать в текущем варианте исследования, т.к. не видна связь, с тем как это может повлиять на главные критерии - семейное положение, кол-во детей и другие. Все остальные столбцы оставляем.

Для осуществления задуманного и для удобства, мы выделим необходимые для нашей работы данные(столбцы) в отдельный dataframe и будем работать уже с ними.

In [20]:
#выделяем необходимые нам для работы, данные в отдельный dataframe
df = df[['gender', 'children', 'family_status', 'family_status_id', 'income_type', 'debt', 'education', 'education_id', 'total_income', 'purpose']].copy()
df.head()#выведим первые 5 строк нового dataframe

,gender,children,family_status,family_status_id,income_type,debt,education,education_id,total_income,purpose
0,f,1,женат / замужем,0,сотрудник,0,высшее,0,253875,покупка жилья
1,f,1,женат / замужем,0,сотрудник,0,среднее,1,112080,приобретение автомобиля
2,m,0,женат / замужем,0,сотрудник,0,среднее,1,145885,покупка жилья
3,m,3,женат / замужем,0,сотрудник,0,среднее,1,267628,дополнительное образование
4,f,0,гражданский брак,1,пенсионер,0,среднее,1,158616,сыграть свадьбу


**Вывод**

Все получилось на отлично. Дубликаты мы удалили, текстовые строки привели в порядок.

### Лемматизация

Импортируем модуль Mystem библиотеки pymystem3 Запись внесем наверху в самую первую строку проекта, согласно рекомендациям Pep8

In [21]:
m = Mystem() #сохраним функцию Mystem в переменную m

def lemma(text): # пишем функцию для лемматизации текста ячейки
    text_obr = m.lemmatize(text) # применим метод m.lemmatize к тексту и сохраним в text_obr
    return ''.join(text_obr).replace('\n', '') #

df['purpose_lemma'] = df['purpose'].apply(lemma) #применим нашу функцию и запишем в отдельный столбец


display(df.head()) #выведим получившийся dataframe

,gender,children,family_status,family_status_id,income_type,debt,education,education_id,total_income,purpose,purpose_lemma
0,f,1,женат / замужем,0,сотрудник,0,высшее,0,253875,покупка жилья,покупка жилье
1,f,1,женат / замужем,0,сотрудник,0,среднее,1,112080,приобретение автомобиля,приобретение автомобиль
2,m,0,женат / замужем,0,сотрудник,0,среднее,1,145885,покупка жилья,покупка жилье
3,m,3,женат / замужем,0,сотрудник,0,среднее,1,267628,дополнительное образование,дополнительный образование
4,f,0,гражданский брак,1,пенсионер,0,среднее,1,158616,сыграть свадьбу,сыграть свадьба


**Вывод**

Мы успешно провели лемматизацию. Создали отдельный столбец для лемматизированных данных. В дальнейшем мы сможем использовать эти данные для категоризации.

### Категоризация данных

Последовательно выполним категоризацию данных по столбцам

#### Категоризация данных по целям кредита

Одним из методов для подсчета количества встречаемости значений, можем применить метод Counter. Так и сделаем, импортируем метод Counter из библиотеки collections. Запись внесем в первую строку проекта, согласно рекомендациям Pep8

Применим метод Counter к списку значений столбца *purpose_lemma*, для того чтобы понять сколько и каких лемматизированных значений в столце.

In [22]:
#используем метод Counter() выведим все значения столбца purpose_lemma с кол-во значений
print(Counter(df['purpose_lemma']))  

Counter({'автомобиль': 972, 'свадьба': 791, 'на проведение свадьба': 768, 'сыграть свадьба': 765, 'операция с недвижимость': 675, 'покупка коммерческий недвижимость': 661, 'операция с жилье': 652, 'покупка жилье для сдача': 651, 'операция с коммерческий недвижимость': 650, 'покупка жилье': 646, 'жилье': 646, 'покупка жилье для семья': 638, 'строительство собственный недвижимость': 635, 'недвижимость': 633, 'операция со свой недвижимость': 627, 'строительство жилой недвижимость': 624, 'покупка недвижимость': 621, 'покупка свой жилье': 620, 'строительство недвижимость': 619, 'ремонт жилье': 607, 'покупка жилой недвижимость': 606, 'на покупка свой автомобиль': 505, 'заниматься высокий образование': 496, 'сделка с подержанный автомобиль': 486, 'на покупка подержать автомобиль': 478, 'свой автомобиль': 478, 'на покупка автомобиль': 471, 'приобретение автомобиль': 461, 'дополнительный образование': 460, 'сделка с автомобиль': 455, 'высокий образование': 452, 'образование': 447, 'получение до

Как мы видим, из списка явно выделяются следующие общие составляющие: автомобиль, свадьба, недвижимость/жилье, образование. Каждый из них составляет свою смысловую группу. В связи с этим, мы можем сформировать/объединить в следующие группы/категории:
1. На автомобиль
2. На свадьбу
3. На недвижимость
4. На образование

Категории получились краткие, и в тоже время понятные.

Чтобы применить данные категории, для этого напишем функцию по замене текущих данных на новые категории и применим к столбцу *purpose_lemma*.

In [23]:
#создадим функцию для замены значений в столбце purpose_lemma на созданные нами категории
def purpose_group_change(row): 
    if 'автомобиль' in row['purpose_lemma']: #для этого применим условия
        return 'на автомобиль' 
    elif 'свадьба' in row['purpose_lemma']:
        return 'на свадьбу'
    elif 'недвижимость' in row['purpose_lemma'] or 'жилье' in row['purpose_lemma']:
        return 'на недвижимость'
    else:
        return 'на образование'

In [24]:
#используя метод apply() применим нашу функцию purpose_group_change для столбца purpose_lemma
df['purpose_lemma'] = df.apply(purpose_group_change, axis=1) 

In [25]:
print(Counter(df['purpose_lemma'])) #выведим для проверки сформированные нами категории

Counter({'на недвижимость': 10811, 'на автомобиль': 4306, 'на образование': 4013, 'на свадьбу': 2324})


Мы успешно категоризировали данные о целях кредита.

#### Категоризация данных по наличию детей

Проработаем столбец с данными наличия детей. Для начала посмотрим, какие обозначения содержит столбец и их количество.

In [26]:
#выведим уникальные значения по столбцу children с помощью методов groupby() и count()
df.groupby('children')['children'].count()

children
-1        47
 0     14091
 1      4808
 2      2052
 3       330
 4        41
 5         9
 20       76
Name: children, dtype: int64

Мы видим что в столбце *children* есть не корректно заполненые значения -1 и 20. Можно предположить, что данные записи были введены не правильно при введении значений 1 (значение -1) и 2 (значение 20). Т.к. эти не корректные значения, составляют незначительную часть от предполагаемых групп, и существенно не исказят итоговые значения, скорректируем их и внесем в эти группы.

In [27]:
df['children'] = df['children'].replace(-1, 1) #заменим с помощью метода replace() -1 на 1
df['children'] = df['children'].replace(20, 2) #заменим с помощью метода replace() 20 на 2

In [28]:
df['children'].describe()

count    21454.000000
mean         0.480563
std          0.756069
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          5.000000
Name: children, dtype: float64

In [29]:
#для проверки, все ли правильно сделали, выведим уникальные значения по столбцу children 
#с помощью метода value_counts()
df['children'].value_counts()

0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

В связи с тем, что значения с 1 и 2 детьми каждое само по себе значительно, а также в связи с тем, что это один из основных запросов заказчика на проверку, не будем объединять их в одну группу. 

Кредиторов с 3 и более детьми относительно не так много, по сравнению с другими, их объединим в одну группу. 

Создадим следующие категории:
* 0 - без детей
* 1 - с 1 ребенком
* 2 - с 2 детьми
* 3 и более - многодетные

In [30]:
#создадим функцию для замены значений в столбце purpose_lemma на созданные нами категории
def children_group_change(row): 
    if row['children'] == 0: #для этого применим условия
        return 'без детей' 
    elif row['children'] == 1:
        return 'с 1 ребенком'
    elif row['children'] == 2:
        return 'с 2 детьми'
    else:
        return 'многодетные'

In [31]:
#применим функцию children_group_change по назначению
df['children_group'] = df.apply(children_group_change, axis=1)

Мы успешно категоризировали данные о наличие детей.

#### Категоризация данных по наличию долга

В условиях задания, нам не были указанны, какие значения колонки *debt* что означают. Что означает значение 1, а что 0? Узнать у заказчиков мы не можем. 

Следовательно, будем исходить из логики и оптимизма, о том каких результатов будет меньше, это и будет означать **наличие долга**.

In [32]:
df['debt'].value_counts() #выведим кол-во значений 0 и 1 с помощью метода value_counts()

0    19713
1     1741
Name: debt, dtype: int64

Мы видим, что значений 0 - 19027, а 1 - 1728. Соответственно, 0 - примем за отсутствие долга, 1 - за наличие долга. Исходя из этого создадим две категории:
* 1 - "с просрочкой"
* 0 - "без просрочек"

Напишем функцию для создания этих категорий.

In [33]:
def debt_group_change(row): #создадим функцию для замены значений в столбце purpose_lemma на созданные нами категории
    if row['debt'] == 1: #для этого применим условия
        return 'с просрочкой' 
    else:
        return 'без просрочек'

In [34]:
#применим функцию debt_group_change по назначению
df['debt_group'] = df.apply(debt_group_change, axis=1)

In [35]:
df.head()

,gender,children,family_status,family_status_id,income_type,debt,education,education_id,total_income,purpose,purpose_lemma,children_group,debt_group
0,f,1,женат / замужем,0,сотрудник,0,высшее,0,253875,покупка жилья,на недвижимость,с 1 ребенком,без просрочек
1,f,1,женат / замужем,0,сотрудник,0,среднее,1,112080,приобретение автомобиля,на автомобиль,с 1 ребенком,без просрочек
2,m,0,женат / замужем,0,сотрудник,0,среднее,1,145885,покупка жилья,на недвижимость,без детей,без просрочек
3,m,3,женат / замужем,0,сотрудник,0,среднее,1,267628,дополнительное образование,на образование,многодетные,без просрочек
4,f,0,гражданский брак,1,пенсионер,0,среднее,1,158616,сыграть свадьбу,на свадьбу,без детей,без просрочек


Мы успешно категоризировали данные о наличие долга.

#### Категоризация данных по уровню дохода

Для категоризации колонки с доходами *total_income* будем использовать квантили (0.25, 0.5(медиана), 0.75). На основе этого создадим следующие категории:
* до 0.25 - низкие доходы
* до 0.5(медианы) - средние доходы
* до 0.75 - выше среднего доходы
* свыше 0.75 - высокие доходы

In [36]:
# #создадим функцию для замены значений в столбце purpose_lemma на созданные нами категории
# def total_income_group_change(row): 
#     if row['total_income'] <= df['total_income'].quantile(0.25): #для этого применим условия
#         return 'низкие доходы' 
#     elif row['total_income'] <= df['total_income'].quantile(0.50):
#         return 'средние доходы'
#     elif row['total_income'] <= df['total_income'].quantile(0.75):
#         return 'выше среднего доходы'
#     else:
#         return 'высокие доходы'

Применим к нашему dataframe созданные функции и создадим отдельный столбец

In [37]:
#применим функцию total_income_group_change по назначению
#df['total_income_group'] = df.apply(total_income_group_change, axis=1)
df['total_income_group'] = pd.qcut(df['total_income'], 4, labels = ['низкие доходы', 'средние доходы', 'выше среднего доходы', 'высокие доходы'])

In [38]:
df.head()

,gender,children,family_status,family_status_id,income_type,debt,education,education_id,total_income,purpose,purpose_lemma,children_group,debt_group,total_income_group
0,f,1,женат / замужем,0,сотрудник,0,высшее,0,253875,покупка жилья,на недвижимость,с 1 ребенком,без просрочек,высокие доходы
1,f,1,женат / замужем,0,сотрудник,0,среднее,1,112080,приобретение автомобиля,на автомобиль,с 1 ребенком,без просрочек,средние доходы
2,m,0,женат / замужем,0,сотрудник,0,среднее,1,145885,покупка жилья,на недвижимость,без детей,без просрочек,выше среднего доходы
3,m,3,женат / замужем,0,сотрудник,0,среднее,1,267628,дополнительное образование,на образование,многодетные,без просрочек,высокие доходы
4,f,0,гражданский брак,1,пенсионер,0,среднее,1,158616,сыграть свадьбу,на свадьбу,без детей,без просрочек,выше среднего доходы


Мы успешно категоризировали данные по величине дохода.

#### Данные по половому признаку

Проверим какие категории и сколько представлены в столбце gender

In [39]:
df['gender'].value_counts() #посчитаем все значения по составу колонки используя метод value_counts()

f      14174
m       7279
xna        1
Name: gender, dtype: int64

Возникла неожиданная запись zna. Данная запись могла возникнуть по разным причинам, или не правильно внесли данные при заполнении, либо клиент не позиционировал себя c традиционными записями. Т.к. такая запись только одна, то вполне мы можем ею пренебречь и удалить. Что и сделаем с использованием метода drop() по индексу строки под №10462

In [40]:
#выявим по какому индексу расположен этот гендерный признак и какие данные под ним находятся
d = df[df['gender'] == 'xna']
display(d) #выведим на экран

,gender,children,family_status,family_status_id,income_type,debt,education,education_id,total_income,purpose,purpose_lemma,children_group,debt_group,total_income_group
10684,xna,0,гражданский брак,1,компаньон,0,неоконченное высшее,2,203905,покупка недвижимости,на недвижимость,без детей,без просрочек,высокие доходы


In [41]:
#удаляем данную запись с помощью метода drop() по индексу
df = df.drop(index=10462, axis=0)

In [42]:
df.groupby('gender')['gender'].count() #проверим удалилась ли у нас данная строка

gender
f      14173
m       7279
xna        1
Name: gender, dtype: int64

Мы успешно привели в порядок данные по половому признаку.

**Вывод**

In [43]:
#выведим первые 5 строк и посмотрим все ли добавились данные, те что мы хотели добавить
df.head()

,gender,children,family_status,family_status_id,income_type,debt,education,education_id,total_income,purpose,purpose_lemma,children_group,debt_group,total_income_group
0,f,1,женат / замужем,0,сотрудник,0,высшее,0,253875,покупка жилья,на недвижимость,с 1 ребенком,без просрочек,высокие доходы
1,f,1,женат / замужем,0,сотрудник,0,среднее,1,112080,приобретение автомобиля,на автомобиль,с 1 ребенком,без просрочек,средние доходы
2,m,0,женат / замужем,0,сотрудник,0,среднее,1,145885,покупка жилья,на недвижимость,без детей,без просрочек,выше среднего доходы
3,m,3,женат / замужем,0,сотрудник,0,среднее,1,267628,дополнительное образование,на образование,многодетные,без просрочек,высокие доходы
4,f,0,гражданский брак,1,пенсионер,0,среднее,1,158616,сыграть свадьбу,на свадьбу,без детей,без просрочек,выше среднего доходы


Действительно, все впорядке. Все наши функции и категории успешно применились. Присутствуют в dataframe

Мы успешно провели категоризацию по всем необходимым для работы столбцам. 

Проверили данные в них и привели их в порядок. 

Все в порядке, можем приступать к дальнейшей работе.

## Шаг 3. Ответы на вопросы

Прежде чем ответить на вопрос, влияют ли определенные признаки на возврат кредита в срок, нужно определиться, что будет "отправной точкой" выше которой мы будем считать есть ли зависимость или нет от определенных признаков, и в какую сторону - положительную или отрицательную. Этой "точкой" будет средний уровень просрочек платежа.

Вычислим средний показатель просрочек, так называемую "среднюю точку", путем деления всех просрочек на общее количество займов.

In [44]:
avg_debt = df[df['debt'] == 1]['debt'].count() / df['debt'].count() * 100

In [45]:
print(f'Средний уровень просрочек платежа - {round(avg_debt,2)}%')

Средний уровень просрочек платежа - 8.12%


При формировании выводов будем использовать/сравнивать с данным значением.

Также сформируем категории для обозначения ситуации по отклонениям от средней величины. Это нужно для того, чтобы мы и наши заказчики сразу видели те категории и комбинации, на которые стоит обратить самое пристальное влияние и к каким стремиться.
Сформируем следующие категории:
* превышение от среднего уровня просрочек в 2 и более раза - "Критический случай!"
* превышение от среднего уровня просрочек на 50% и более до 2 раз - "Особая проблемная ситуация!"
* превышение от среднего уровня просрочек от средней и до 50% - "Проблемная ситуация! На заметку"
* положительное отклонение от среднего уровня от средней - "Хорошая ситуация"
* значения в которых не было просрочек, либо только просрочки, обозначаются в таблицах, как NaN, следовательно - "Крайние значения"

На "Крайние значение" стоит в любом случае (как в положительном, так и отрицатльном) обращать особое внимание.

Для этого создадим функцию которая будет обозначать нам эти категории в таблице.

In [46]:
def category_special_case(data):
    """Функция категоризирует отклонения от среднего значения"""
    
    #в сводных таблицах, к необходимым ячейкам, можно обратиться через мультииндексы
    for row in list(data.index): #перебераем комбинации вариантов
        if data.loc[row, 'deviation'] <= -avg_debt: #категоризируем варианты с исользованием условий
            data.loc[row, 'category_special_case'] = 'Критический случай!'
        elif data.loc[row, 'deviation'] <= -avg_debt / 2:
            data.loc[row, 'category_special_case'] = 'Особая проблемная ситуация!'
        elif data.loc[row, 'deviation'] <= 0:
            data.loc[row, 'category_special_case'] = 'Проблемная ситуация.'
        elif data.loc[row, 'deviation'] > 0:
            data.loc[row, 'category_special_case'] = 'Хорошая ситуация'
        else:
            data.loc[row, 'category_special_case'] = 'Крайние значения' 
        

### Зависимости

#### Зависимость с учетом наличия детей

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Сформируем сводную таблицу, в которой будут отражаться данные зависимости наличия просрочек от наличия детей.

In [47]:
#сформируем сводную таблицу
data_pivot_children = pd.pivot_table(df, index = ['children_group'], \
                            columns = 'debt_group', values = 'debt', aggfunc = 'count')

#добавим столбец относительной величины просрочек к общим кредитам
data_pivot_children['avg_debt_%'] = round(data_pivot_children['с просрочкой'] \
            / (data_pivot_children['с просрочкой'] + data_pivot_children['без просрочек']) * 100, 2)

#добавим столбец, в котором покажем с отклонениями от среднего показателия
data_pivot_children['deviation'] = round(avg_debt - data_pivot_children['avg_debt_%'], 2)

#добавим столбец, показывающий на сколько % отклонение от средней величины
data_pivot_children['deviation_%'] = round(data_pivot_children['deviation'] / data_pivot_children['avg_debt_%'] * 100, 1)

#применим функцию категоризации отклонений от среднего показателия
category_special_case(data_pivot_children)

#выведим таблицу на экран и одновременно отсортируем по столбцу отклонений от среднего
display(data_pivot_children.sort_values('deviation', ascending=True))

debt_group,без просрочек,с просрочкой,avg_debt_%,deviation,deviation_%,category_special_case
children_group,,,,,,
с 2 детьми,1926,202,9.49,-1.37,-14.4,Проблемная ситуация.
с 1 ребенком,4410,445,9.17,-1.05,-11.5,Проблемная ситуация.
многодетные,349,31,8.16,-0.04,-0.5,Проблемная ситуация.
без детей,13027,1063,7.54,0.58,7.7,Хорошая ситуация


Как видим из таблицы, при применении только одного вида категории данных, а в данном случае это по наличию детей, отклонения от среднего значения в районе 12-13% в проблемную сторону и до 8% в положительную сторону.

Сделаем комбинации с другими категориями и посмотрим - меняются ли отклонения, есть ли зависимости в комбинациях. Применим дополнительно к категории по наличию детей, по очереди следующие категории:
* пол
* семейный статус
* рабочий статус
* образование 
* цель кредита
* уровень доходов

Создадим функцию в которой будем перебирать данные вариации и выводить их на экран. В дальнейшем, она нам также пригодиться для других комбинаций-зависимостей.

In [48]:
def create_combination(main_group, lst):
    '''Функция перебирает список дополнительных зависимостей lst и 
    комбинирует их с основной зависимостью main_group'''
    
    for addiction in lst:   #перебираем список дополнительных зависимостей/категорий
        
        #выводим обозначение комбинации
        print(f'Зависимость категорий {main_group} и {addiction}:')
        
        #сформировываем сводную таблицу
        data_pivot = pd.pivot_table(df, index = [main_group, addiction], \
                            columns = 'debt_group', values = 'debt', aggfunc = 'count')
        
        #добавляем столбец с относительными показателями просрочки платежа
        data_pivot['avg_debt_%'] = round(data_pivot['с просрочкой'] \
            / (data_pivot['с просрочкой'] + data_pivot['без просрочек']) * 100, 2)

        #добавляем столбец с отклонениями от средней величины
        data_pivot['deviation'] = round(avg_debt - data_pivot['avg_debt_%'], 2)
        
        #добавим столбец, показывающий на сколько % отклонение от средней величины
        data_pivot['deviation_%'] = round(data_pivot['deviation'] / data_pivot['avg_debt_%'] * 100, 1)

        #применим функцию категоризации отклонений от среднего показателия
        category_special_case(data_pivot)

        #выводим сводную таблицу на экран
        display(data_pivot.sort_values('deviation', ascending=True))

In [49]:
#создаем лист из возможных дополнительных комбинаций
list_addiction_with_children_group = ['gender', 'family_status', 'income_type', 'education', 'purpose_lemma', 'total_income_group']

In [50]:
#применим функцию для  комбинации основной категории с одной дополнительной
create_combination('children_group', list_addiction_with_children_group)

Зависимость категорий children_group и gender:


debt_group             без просрочек  с просрочкой  avg_debt_%  deviation  \
children_group gender                                                       
с 1 ребенком   m              1536.0         199.0       11.47      -3.35   
с 2 детьми     f              1165.0         138.0       10.59      -2.47   
без детей      m              4099.0         471.0       10.31      -2.19   
многодетные    m               136.0          13.0        8.72      -0.60   
с 1 ребенком   f              2874.0         246.0        7.88       0.24   
многодетные    f               213.0          18.0        7.79       0.33   
с 2 детьми     m               761.0          64.0        7.76       0.36   
без детей      f              8927.0         592.0        6.22       1.90   
               xna               1.0           NaN         NaN        NaN   

debt_group             deviation_% category_special_case  
children_group gender                                     
с 1 ребенком   m             -29.2  Проблемная ситуация.  
с 2 детьми     f             -23.3  Проблемная ситуация.  
без детей      m             -21.2  Проблемная ситуация.  
многодетные    m              -6.9  Проблемная ситуация.  
с 1 ребенком   f               3.0      Хорошая ситуация  
многодетные    f               4.2      Хорошая ситуация  
с 2 детьми     m               4.6      Хорошая ситуация  
без детей      f              30.5      Хорошая ситуация  
               xna             NaN      Крайние значения

Зависимость категорий children_group и family_status:


debt_group                            без просрочек  с просрочкой  avg_debt_%  \
children_group family_status                                                    
многодетные    не женат / не замужем            8.0           2.0       20.00   
с 2 детьми     вдовец / вдова                  21.0           3.0       12.50   
многодетные    гражданский брак                58.0           8.0       12.12   
с 2 детьми     не женат / не замужем           74.0          10.0       11.90   
с 1 ребенком   гражданский брак               882.0         118.0       11.80   
               не женат / не замужем          402.0          52.0       11.45   
с 2 детьми     в разводе                       75.0           8.0        9.64   
               женат / замужем               1434.0         148.0        9.36   
               гражданский брак               322.0          33.0        9.30   
без детей      не женат / не замужем         2052.0         210.0        9.28   
с 1 ребенком   вдовец / вдова                  74.0           7.0        8.64   
без детей      гражданский брак              2500.0         229.0        8.39   
многодетные    в разводе                       11.0           1.0        8.33   
с 1 ребенком   женат / замужем               2757.0         247.0        8.22   
без детей      в разводе                      729.0          55.0        7.02   
многодетные    женат / замужем                265.0          20.0        7.02   
без детей      женат / замужем               6952.0         516.0        6.91   
с 1 ребенком   в разводе                      295.0          21.0        6.65   
без детей      вдовец / вдова                 794.0          53.0        6.26   
многодетные    вдовец / вдова                   7.0           NaN         NaN   

debt_group                            deviation  deviation_%  \
children_group family_status                                   
многодетные    не женат / не замужем     -11.88        -59.4   
с 2 детьми     вдовец / вдова             -4.38        -35.0   
многодетные    гражданский брак           -4.00        -33.0   
с 2 детьми     не женат / не замужем      -3.78        -31.8   
с 1 ребенком   гражданский брак           -3.68        -31.2   
               не женат / не замужем      -3.33        -29.1   
с 2 детьми     в разводе                  -1.52        -15.8   
               женат / замужем            -1.24        -13.2   
               гражданский брак           -1.18        -12.7   
без детей      не женат / не замужем      -1.16        -12.5   
с 1 ребенком   вдовец / вдова             -0.52         -6.0   
без детей      гражданский брак           -0.27         -3.2   
многодетные    в разводе                  -0.21         -2.5   
с 1 ребенком   женат / замужем            -0.10         -1.2   
без детей      в разводе                   1.10         15.7   
многодетные    женат / замужем             1.10         15.7   
без детей      женат / замужем             1.21         17.5   
с 1 ребенком   в разводе                   1.47         22.1   
без детей      вдовец / вдова              1.86         29.7   
многодетные    вдовец / вдова               NaN          NaN   

debt_group                                  category_special_case  
children_group family_status                                       
многодетные    не женат / не замужем          Критический случай!  
с 2 детьми     вдовец / вдова         Особая проблемная ситуация!  
многодетные    гражданский брак              Проблемная ситуация.  
с 2 детьми     не женат / не замужем         Проблемная ситуация.  
с 1 ребенком   гражданский брак              Проблемная ситуация.  
               не женат / не замужем         Проблемная ситуация.  
с 2 детьми     в разводе                     Проблемная ситуация.  
               женат / замужем               Проблемная ситуация.  
               гражданский брак              Проблемная ситуация.  
без детей      не женат / не замужем         Проблемная ситу

Зависимость категорий children_group и income_type:


...

,debt_group,без просрочек,с просрочкой,avg_debt_%,deviation,deviation_%,category_special_case
children_group,education,,,,,,
многодетные,неоконченное высшее,6.0,2.0,25.00,-16.88,-67.5,Критический случай!
с 2 детьми,начальное,19.0,6.0,24.00,-15.88,-66.2,Критический случай!
с 1 ребенком,начальное,50.0,10.0,16.67,-8.55,-51.3,Критический случай!
с 2 детьми,неоконченное высшее,65.0,9.0,12.16,-4.04,-33.2,Проблемная ситуация.
с 1 ребенком,неоконченное высшее,156.0,20.0,11.36,-3.24,-28.5,Проблемная ситуация.
с 2 детьми,среднее,1302.0,158.0,10.82,-2.70,-25.0,Проблемная ситуация.
с 1 ребенком,среднее,2945.0,343.0,10.43,-2.31,-22.1,Проблемная ситуация.
многодетные,среднее,246.0,25.0,9.23,-1.11,-12.0,Проблемная ситуация.
без детей,среднее,9314.0,838.0,8.25,-0.13,-1.6,Проблемная ситуация.


Зависимость категорий children_group и purpose_lemma:


debt_group                      без просрочек  с просрочкой  avg_debt_%  \
children_group purpose_lemma                                              
многодетные    на свадьбу                  31             5       13.89   
с 2 детьми     на автомобиль              365            50       12.05   
               на образование             371            47       11.24   
с 1 ребенком   на автомобиль              870           104       10.68   
               на образование             786            90       10.27   
               на свадьбу                 483            51        9.55   
без детей      на образование            2413           229        8.67   
               на автомобиль             2601           243        8.54   
с 2 детьми     на недвижимость            981            90        8.40   
многодетные    на автомобиль               66             6        8.33   
               на недвижимость            179            16        8.21   
с 1 ребенком   на недвижимость           2271           200        8.09   
без детей      на свадьбу                1415           115        7.52   
               на недвижимость           6598           476        6.73   
с 2 детьми     на свадьбу                 209            15        6.70   
многодетные    на образование              73             4        5.19   

debt_group                      deviation  deviation_%  \
children_group purpose_lemma                             
многодетные    на свадьбу           -5.77        -41.5   
с 2 детьми     на автомобиль        -3.93        -32.6   
               на образование       -3.12        -27.8   
с 1 ребенком   на автомобиль        -2.56        -24.0   
               на образование       -2.15        -20.9   
               на свадьбу           -1.43        -15.0   
без детей      на образование       -0.55         -6.3   
               на автомобиль        -0.42         -4.9   
с 2 детьми     на недвижимость      -0.28         -3.3   
многодетные    на автомобиль        -0.21         -2.5   
               на недвижимость      -0.09         -1.1   
с 1 ребенком   на недвижимость       0.03          0.4   
без детей      на свадьбу            0.60          8.0   
               на недвижимость       1.39         20.7   
с 2 детьми     на свадьбу            1.42         21.2   
многодетные    на образование        2.93         56.5   

debt_group                            category_special_case  
children_group purpose_lemma                                 
многодетные    на свадьбу       Особая проблемная ситуация!  
с 2 детьми     на автомобиль           Проблемная ситуация.  
               на образование          Проблемная ситуация.  
с 1 ребенком   на автомобиль           Проблемная ситуация.  
               на образование          Проблемная ситуация.  
               на свадьбу              Проблемная ситуация.  
без детей      на образование          Проблемная ситуация.  
               на автомобиль           Проблемная ситуация.  
с 2 детьми     на недвижимость         Проблемная ситуация.  
многодетные    на автомобиль           Проблемная ситуация.  
               на недвижимость         Проблемная ситуация.  
с 1 ребенком   на недвижимость             Хорошая ситуация  
без детей      на свадьбу                  Хорошая ситуация  
               на недвижимость             Хорошая ситуация  
с 2 детьми     на свадьбу                  Хорошая ситуация  
многодетные    на образование              Хорошая ситуация

Зависимость категорий children_group и total_income_group:


debt_group                           без просрочек  с просрочкой  avg_debt_%  \
children_group total_income_group                                              
многодетные    низкие доходы                    67             9       11.84   
с 2 детьми     средние доходы                  483            56       10.39   
               выше среднего доходы            450            52       10.36   
с 1 ребенком   средние доходы                 1110           120        9.76   
               выше среднего доходы           1131           121        9.66   
               низкие доходы                  1006           107        9.61   
с 2 детьми     низкие доходы                   486            49        9.16   
многодетные    выше среднего доходы             87             8        8.42   
               средние доходы                   98             9        8.41   
без детей      средние доходы                 3305           298        8.27   
с 2 детьми     высокие доходы                  507            45        8.15   
без детей      выше среднего доходы           3131           267        7.86   
с 1 ребенком   высокие доходы                 1163            97        7.70   
без детей      низкие доходы                  3378           262        7.20   
               высокие доходы                 3213           236        6.84   
многодетные    высокие доходы                   97             5        4.90   

debt_group                           deviation  deviation_%  \
children_group total_income_group                             
многодетные    низкие доходы             -3.72        -31.4   
с 2 детьми     средние доходы            -2.27        -21.8   
               выше среднего доходы      -2.24        -21.6   
с 1 ребенком   средние доходы            -1.64        -16.8   
               выше среднего доходы      -1.54        -15.9   
               низкие доходы             -1.49        -15.5   
с 2 детьми     низкие доходы             -1.04        -11.4   
многодетные    выше среднего доходы      -0.30         -3.6   
               средние доходы            -0.29         -3.4   
без детей      средние доходы            -0.15         -1.8   
с 2 детьми     высокие доходы            -0.03         -0.4   
без детей      выше среднего доходы       0.26          3.3   
с 1 ребенком   высокие доходы             0.42          5.5   
без детей      низкие доходы              0.92         12.8   
               высокие доходы             1.28         18.7   
многодетные    высокие доходы             3.22         65.7   

debt_group                          category_special_case  
children_group total_income_group                          
многодетные    низкие доходы         Проблемная ситуация.  
с 2 детьми     средние доходы        Проблемная ситуация.  
               выше среднего доходы  Проблемная ситуация.  
с 1 ребенком   средние доходы        Проблемная ситуация.  
               выше среднего доходы  Проблемная ситуация.  
               низкие доходы         Проблемная ситуация.  
с 2 детьми     низкие доходы         Проблемная ситуация.  
многодетные    выше среднего доходы  Проблемная ситуация.  
               средние доходы        Проблемная ситуация.  
без детей      средние доходы        Проблемная ситуация.  
с 2 детьми     высокие доходы        Проблемная ситуация.  
без детей      выше среднего доходы      Хорошая ситуация  
с 1 ребенком   высокие доходы            Хорошая ситуация  
без детей      низкие доходы             Хорошая ситуация  
               высокие доходы            Хорошая ситуация  
многодетные    высокие доходы            Хорошая ситуация

Отметим, что при комбинациях категорий "по наличию детей" с "семейным статусом" и с "образованием" есть критические случаи. Особые проблемные ситуации в комбинациях с "семейным статусом", "рабочим статусом" и "целью кредита". 

Положительные крайние ситуации так же есть в комбинациях с "семейным статусом", "рабочим статусом" и "образованием"

!Стоит заметить, все значительные отклонения, так и крайние случаи, как в положительную, так и отрицательную сторону, построены на малой выборке.

Применение комбинаций из 2х параметров, показало свою успешность и эффективность. Для расширения поля исследований, создадим функцию с формированием сводных таблиц из 3 комбинаций - одна основная и 2 дополнительных.

In [51]:
def create_combination_twice(main_group, lst):
    '''Функция перебирает список дополнительных зависимостей lst и комбинирует их с основной зависимостью main_group'''
    
    for addiction_first in lst[:round(len(lst)/2)]:#перебираем список дополнительных зависимостей/категорий
        for addiction_second in lst[round(len(lst)/2):]:
        
            print(f'Зависимость категорий {main_group} с {addiction_first} и {addiction_second}:') #выводим обозначение комбинации
        
            #сформировываем сводную таблицу
            data_pivot = pd.pivot_table(df, index = [main_group, addiction_first, addiction_second], \
                            columns = 'debt_group', values = 'debt', aggfunc = 'count')
        
            #добавляем столбец с относительными показателями просрочки платежа
            data_pivot['avg_debt_%'] = round(data_pivot['с просрочкой'] \
                / (data_pivot['с просрочкой'] + data_pivot['без просрочек']) * 100, 2)

            #добавляем столбец с отклонениями от средней величины
            data_pivot['deviation'] = round(avg_debt - data_pivot['avg_debt_%'], 2)
            
            
            #добавим столбец, показывающий на сколько % отклонение от средней величины
            data_pivot['deviation_%'] = round(data_pivot['deviation'] / data_pivot['avg_debt_%'] * 100, 1)
            
            
            #применим функцию категоризации отклонений от среднего показателия
            category_special_case(data_pivot)
            
            #выводим сводную таблицу на экран
            display(data_pivot.sort_values('deviation', ascending=True))

In [52]:
#применим функцию для комбинации основной категории с двумя дополнительными
create_combination_twice('children_group', list_addiction_with_children_group)

Зависимость категорий children_group с gender и education:


debt_group                                 без просрочек  с просрочкой  \
children_group gender education                                          
с 2 детьми     f      начальное                      7.0           4.0   
многодетные    m      неоконченное высшее            2.0           1.0   
с 1 ребенком   m      начальное                     23.0           6.0   
многодетные    f      неоконченное высшее            4.0           1.0   
с 2 детьми     f      неоконченное высшее           37.0           7.0   
               m      начальное                     12.0           2.0   
с 1 ребенком   m      среднее                     1077.0         160.0   
               f      начальное                     27.0           4.0   
с 2 детьми     f      среднее                      756.0         109.0   
с 1 ребенком   m      неоконченное высшее           52.0           7.0   
без детей      m      среднее                     2849.0         374.0   
с 1 ребенком   f      неоконченное высшее          104.0          13.0   
многодетные    m      среднее                       96.0          11.0   
без детей      m      начальное                     64.0           7.0   
с 1 ребенком   f      среднее                     1868.0         183.0   
без детей      m      неоконченное высшее          174.0          17.0   
многодетные    f      среднее                      150.0          14.0   
с 2 детьми     m      среднее                      546.0          49.0   
без детей      f      неоконченное высшее          274.0          20.0   
                      начальное                    110.0           8.0   
               m      высшее                      1009.0          73.0   
               f      среднее                     6465.0         464.0   
с 2 детьми     m      неоконченное высшее           28.0           2.0   
с 1 ребенком   m      высшее                       383.0          26.0   
с 2 детьми     m      высшее                       175.0          11.0   
многодетные    f      высшее                        55.0           3.0   
с 1 ребенком   f      высшее                       875.0          46.0   
с 2 детьми     f      высшее                       365.0          18.0   
без детей      f      высшее                      2077.0         100.0   
многодетные    m      высшее                        33.0           1.0   
без детей      f      ученая степень                 1.0           NaN   
               m      ученая степень                 3.0           NaN   
               xna    неоконченное высшее            1.0           NaN   
многодетные    f      начальное                      3.0           NaN   
                      ученая степень                 1.0           NaN   
               m      начальное                      5.0           NaN   
с 1 ребенком   m      ученая степень                 1.0           NaN   

debt_group                                 avg_debt_%  deviation  deviation_%  \
children_group gender education                                                 
с 2 детьми     f      начальное                 36.36     -28.24        -77.7   
многодетные    m      неоконченное высшее       33.33     -25.21        -75.6   
с 1 ребенком   m      начальное                 20.69     -12.57        -60.8   
многодетные    f      неоконченное высшее       20.00     -11.88        -59.4   
с 2 детьми     f      неоконченное высшее       15.91      -7.79        -49.0   
               m      начальное                 14.29      -6.17        -43.2   
с 1 ребенком   m      среднее                   12.93      -4.81        -37.2   
               f      начальное                 12.90      -4.78        -37.1   
с 2 детьми     f      среднее                   12.60      -4.48        -35.6   
с 1 ребенком   m      неоконченное высшее       11.86      -3.74        -31.5   
без детей      m      среднее                   11.60      -3.48        -30.0   
с 1 ребенком   f      неоконченное высшее       11.11      -

Зависимость категорий children_group с gender и purpose_lemma:


debt_group                             без просрочек  с просрочкой  \
children_group gender purpose_lemma                                  
многодетные    f      на свадьбу                17.0           4.0   
с 2 детьми     f      на автомобиль            215.0          34.0   
с 1 ребенком   m      на образование           256.0          38.0   
                      на свадьбу               177.0          26.0   
без детей      m      на образование           743.0         108.0   
с 1 ребенком   m      на автомобиль            311.0          45.0   
с 2 детьми     f      на образование           219.0          31.0   
многодетные    m      на автомобиль             22.0           3.0   
без детей      m      на автомобиль            795.0         104.0   
многодетные    m      на недвижимость           68.0           8.0   
с 1 ребенком   m      на недвижимость          792.0          90.0   
без детей      m      на свадьбу               424.0          47.0   
с 2 детьми     m      на автомобиль            150.0          16.0   
с 1 ребенком   f      на автомобиль            559.0          59.0   
с 2 детьми     m      на образование           152.0          16.0   
               f      на недвижимость          591.0          62.0   
без детей      m      на недвижимость         2137.0         212.0   
с 1 ребенком   f      на образование           530.0          52.0   
                      на свадьбу               306.0          25.0   
с 2 детьми     f      на свадьбу               140.0          11.0   
без детей      f      на автомобиль           1806.0         139.0   
с 1 ребенком   f      на недвижимость         1479.0         110.0   
многодетные    f      на образование            41.0           3.0   
без детей      f      на образование          1670.0         121.0   
многодетные    f      на недвижимость          111.0           8.0   
с 2 детьми     m      на недвижимость          390.0          28.0   
многодетные    m      на свадьбу                14.0           1.0   
без детей      f      на свадьбу               991.0          68.0   
многодетные    f      на автомобиль             44.0           3.0   
без детей      f      на недвижимость         4460.0         264.0   
с 2 детьми     m      на свадьбу                69.0           4.0   
многодетные    m      на образование            32.0           1.0   
без детей      xna    на недвижимость            1.0           NaN   

debt_group                             avg_debt_%  deviation  deviation_%  \
children_group gender purpose_lemma                                         
многодетные    f      на свадьбу            19.05     -10.93        -57.4   
с 2 детьми     f      на автомобиль         13.65      -5.53        -40.5   
с 1 ребенком   m      на образование        12.93      -4.81        -37.2   
                      на свадьбу            12.81      -4.69        -36.6   
без детей      m      на образование        12.69      -4.57        -36.0   
с 1 ребенком   m      на автомобиль         12.64      -4.52        -35.8   
с 2 детьми     f      на образование        12.40      -4.28        -34.5   
многодетные    m      на автомобиль         12.00      -3.88        -32.3   
без детей      m      на автомобиль         11.57      -3.45        -29.8   
многодетные    m      на недвижимость       10.53      -2.41        -22.9   
с 1 ребенком   m      на недвижимость       10.20      -2.08        -20.4   
без детей      m      на свадьбу             9.98      -1.86        -18.6   
с 2 детьми     m      на автомобиль          9.64      -1.52        -15.8   
с 1 ребенком   f      на автомобиль          9.55      -1.43        -15.0   
с 2 детьми     m      на образование         9.52      -1.40        -14.7   
               f      на недвижимость        9.49      -1.37        -14.4   
без детей      m      на недвижимость        9.03      -0.91        -10.1   
с 1 ребенком   f      на образование         8.93      -0.81         -9.1   
         

Зависимость категорий children_group с gender и total_income_group:


...

debt_group                                      без просрочек  с просрочкой  \
children_group income_type education                                          
с 2 детьми     сотрудник   начальное                     14.0           6.0   
с 1 ребенком   сотрудник   начальное                     32.0           8.0   
без детей      компаньон   начальное                     13.0           3.0   
с 2 детьми     сотрудник   неоконченное высшее           36.0           8.0   
многодетные    пенсионер   среднее                        5.0           1.0   
...                                                       ...           ...   
с 1 ребенком   сотрудник   ученая степень                 1.0           NaN   
с 2 детьми     в декрете   среднее                        NaN           1.0   
               госслужащий неоконченное высшее            5.0           NaN   
               компаньон   начальное                      4.0           NaN   
               пенсионер   начальное                      1.0           NaN   

debt_group                                      avg_debt_%  deviation  \
children_group income_type education                                    
с 2 детьми     сотрудник   начальное                 30.00     -21.88   
с 1 ребенком   сотрудник   начальное                 20.00     -11.88   
без детей      компаньон   начальное                 18.75     -10.63   
с 2 детьми     сотрудник   неоконченное высшее       18.18     -10.06   
многодетные    пенсионер   среднее                   16.67      -8.55   
...                                                    ...        ...   
с 1 ребенком   сотрудник   ученая степень              NaN        NaN   
с 2 детьми     в декрете   среднее                     NaN        NaN   
               госслужащий неоконченное высшее         NaN        NaN   
               компаньон   начальное                   NaN        NaN   
               пенсионер   начальное                   NaN        NaN   

debt_group                                      deviation_%  \
children_group income_type education                          
с 2 детьми     сотрудник   начальное                  -72.9   
с 1 ребенком   сотрудник   начальное                  -59.4   
без детей      компаньон   начальное                  -56.7   
с 2 детьми     сотрудник   неоконченное высшее        -55.3   
многодетные    пенсионер   среднее                    -51.3   
...                                                     ...   
с 1 ребенком   сотрудник   ученая степень               NaN   
с 2 детьми     в декрете   среднее                      NaN   
               госслужащий неоконченное высшее          NaN   
               компаньон   начальное                    NaN   
               пенсионер   начальное                    NaN   

debt_group                                     category_special_case  
children_group income_type education                                  
с 2 детьми     сотрудник   начальное             Критический случай!  
с 1 ребенком   сотрудник   начальное             Критический случай!  
без детей      компаньон   начальное             Критический случай!  
с 2 детьми     сотрудник   неоконченное высшее   Критический случай!  
многодетные    пенсионер   среднее               Критический случай!  
...                                                              ...  
с 1 ребенком   сотрудник   ученая степень           Крайние значения  
с 2 детьми     в декрете   среднее                  Крайние значения  
               госслужащий неоконченное высшее      Крайние значения  
               компаньон   начальное                Крайние значения  
               пенсионер   начальное                Крайние значения  

[66 rows x 6 columns]

Зависимость категорий children_group с income_type и purpose_lemma:


debt_group                                  без просрочек  с просрочкой  \
children_group income_type purpose_lemma                                  
с 2 детьми     пенсионер   на образование             4.0           2.0   
многодетные    сотрудник   на свадьбу                22.0           4.0   
с 2 детьми     пенсионер   на автомобиль              6.0           1.0   
               сотрудник   на автомобиль            223.0          37.0   
с 1 ребенком   компаньон   на свадьбу               124.0          20.0   
...                                                   ...           ...   
               безработный на недвижимость            NaN           1.0   
с 2 детьми     в декрете   на автомобиль              NaN           1.0   
               госслужащий на свадьбу                24.0           NaN   
               пенсионер   на недвижимость           14.0           NaN   
                           на свадьбу                 2.0           NaN   

debt_group                                  avg_debt_%  deviation  \
children_group income_type purpose_lemma                            
с 2 детьми     пенсионер   на образование        33.33     -25.21   
многодетные    сотрудник   на свадьбу            15.38      -7.26   
с 2 детьми     пенсионер   на автомобиль         14.29      -6.17   
               сотрудник   на автомобиль         14.23      -6.11   
с 1 ребенком   компаньон   на свадьбу            13.89      -5.77   
...                                                ...        ...   
               безработный на недвижимость         NaN        NaN   
с 2 детьми     в декрете   на автомобиль           NaN        NaN   
               госслужащий на свадьбу              NaN        NaN   
               пенсионер   на недвижимость         NaN        NaN   
                           на свадьбу              NaN        NaN   

debt_group                                  deviation_%  \
children_group income_type purpose_lemma                  
с 2 детьми     пенсионер   на образование         -75.6   
многодетные    сотрудник   на свадьбу             -47.2   
с 2 детьми     пенсионер   на автомобиль          -43.2   
               сотрудник   на автомобиль          -42.9   
с 1 ребенком   компаньон   на свадьбу             -41.5   
...                                                 ...   
               безработный на недвижимость          NaN   
с 2 детьми     в декрете   на автомобиль            NaN   
               госслужащий на свадьбу               NaN   
               пенсионер   на недвижимость          NaN   
                           на свадьбу               NaN   

debt_group                                        category_special_case  
children_group income_type purpose_lemma                                 
с 2 детьми     пенсионер   на образование           Критический случай!  
многодетные    сотрудник   на свадьбу       Особая проблемная ситуация!  
с 2 детьми     пенсионер   на автомобиль    Особая проблемная ситуация!  
               сотрудник   на автомобиль    Особая проблемная ситуация!  
с 1 ребенком   компаньон   на свадьбу       Особая проблемная ситуация!  
...                                                                 ...  
               безработный на недвижимость             Крайние значения  
с 2 детьми     в декрете   на автомобиль               Крайние значения  
               госслужащий на свадьбу                  Крайние значения  
               пенсионер   на недвижимость             Крайние значения  
                           на свадьбу                  Крайние значения  

[69 rows x 6 columns]

Зависимость категорий children_group с income_type и total_income_group:


debt_group                                           без просрочек  \
children_group income_type     total_income_group                    
с 2 детьми     в декрете       низкие доходы                     0   
с 1 ребенком   безработный     низкие доходы                     0   
многодетные    пенсионер       низкие доходы                     2   
               госслужащий     низкие доходы                     7   
с 2 детьми     пенсионер       выше среднего доходы              6   
...                                                            ...   
               предприниматель высокие доходы                    0   
               студент         низкие доходы                     0   
                               средние доходы                    0   
                               выше среднего доходы              0   
                               высокие доходы                    0   

debt_group                                           с просрочкой  avg_debt_%  \
children_group income_type     total_income_group                               
с 2 детьми     в декрете       низкие доходы                    1      100.00   
с 1 ребенком   безработный     низкие доходы                    1      100.00   
многодетные    пенсионер       низкие доходы                    1       33.33   
               госслужащий     низкие доходы                    2       22.22   
с 2 детьми     пенсионер       выше среднего доходы             1       14.29   
...                                                           ...         ...   
               предприниматель высокие доходы                   0         NaN   
               студент         низкие доходы                    0         NaN   
                               средние доходы                   0         NaN   
                               выше среднего доходы             0         NaN   
                               высокие доходы                   0         NaN   

debt_group                                           deviation  deviation_%  \
children_group income_type     total_income_group                             
с 2 детьми     в декрете       низкие доходы            -91.88        -91.9   
с 1 ребенком   безработный     низкие доходы            -91.88        -91.9   
многодетные    пенсионер       низкие доходы            -25.21        -75.6   
               госслужащий     низкие доходы            -14.10        -63.5   
с 2 детьми     пенсионер       выше среднего доходы      -6.17        -43.2   
...                                                        ...          ...   
               предприниматель высокие доходы              NaN          NaN   
               студент         низкие доходы               NaN          NaN   
                               средние доходы              NaN          NaN   
                               выше среднего доходы        NaN          NaN   
                               высокие доходы              NaN          NaN   

debt_group                                                 category_special_case  
children_group income_type     total_income_group                                 
с 2 детьми     в декрете       низкие доходы                 Критический случай!  
с 1 ребенком   безработный     низкие доходы                 Критический случай!  
многодетные    пенсионер       низкие доходы                 Критический случай!  
               госслужащий     низкие доходы                 Критический случай!  
с 2 детьми     пенсионер       выше среднего доходы  Особая проблемная ситуация!  
...                                                                          ...  
               предприниматель высокие доходы                   Крайние значения  
               студент         низкие доходы                    Крайние значения  
                               средние доходы                   Крайние значения  
                               выше среднего доходы             Крайние значения  
           

При применении основной категории с 2мя дополнительными, выявились значительные корреляции в данных. Появилось больше критических, особых проблемных ситуаций и крайних случаев. 

Однако важно заметить, серьезные отклонения в основе своей, выявлены на малой выборке. Поэтому принимать или нет текущие данные, это выбор заказчика.

Критические и особые проблемные ситуации, есть во всех комбинациях.

Крайние положительные и крайние отрицательные ситуации есть в комбинациях с "образованием" и "полом", с "семейным положением" и "образованием", с "семейным положением" и "целями", с "семейным положением" и "доходами", с "рабочим положением" и "образованием", с "рабочим состоянием" и "целями", с "рабочим положением" и "доходами"

Мы могли бы и далее увеличивать кол-во комбинаций между категориями, но результаты будут ориентированны еще на меньшей выборке чем с 3мя. А это уже будут не совсем корректные данные.

Поэтому остановимся на комбинации из 3 категорий. А при наличии большей выборки можно будет комбинировать и больше значений.

**Вывод**

Отвечая на вопрос "Есть ли зависимость между наличием детей и возвратом кредита в срок?", можно сказать следующее, есть отклонение/зависимость в худшую сторону у заемщиков с 1м и 2мя детьми на 13,9% и 11,1% соответственно от исходной средней величины. Положительные результаты у категорий "многодетные" на  1,8% и "без детей" на 7,3%.

При группировки с 1м или 2мя дополнительными категориями, есть явные отлонения, включая "критических" и "особых проблемных" ситуаций, но здесь во-первых, нет явных "перекосов" одних категорий по наличию детей от других, и во-вторых, все "критические", "особые" и "крайние" ситуации выявлены на малых выборках. 

Есть явно выраженная зависимость от уровня образования.

#### Зависимость с учетом семейного положения

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [53]:
#сформируем сводную таблицу
data_pivot_family_status = pd.pivot_table(df, index = ['family_status'], \
                            columns = 'debt_group', values = 'debt', aggfunc = 'count')

#добавим столбец относительной величины просрочек к общим кредитам
data_pivot_family_status['avg_debt_%'] = round(data_pivot_family_status['с просрочкой'] \
                          / (data_pivot_family_status['с просрочкой'] + data_pivot_family_status['без просрочек']) * 100, 2)

#добавим столбец, в котором покажем с отклонениями от среднего показателия
data_pivot_family_status['deviation'] = round(avg_debt - data_pivot_family_status['avg_debt_%'], 2)

#добавим столбец, показывающий на сколько % отклонение от средней величины
data_pivot_family_status['deviation_%'] = round(data_pivot_family_status['deviation'] / data_pivot_family_status['avg_debt_%'] * 100, 1)

#применим функцию категоризации отклонений от среднего показателия
category_special_case(data_pivot_family_status)

#выведим таблицу на экран и одновременно отсортируем по столбцу отклонений от среднего
display(data_pivot_family_status.sort_values('deviation', ascending=True))


debt_group,без просрочек,с просрочкой,avg_debt_%,deviation,deviation_%,category_special_case
family_status,,,,,,
не женат / не замужем,2536,274,9.75,-1.63,-16.7,Проблемная ситуация.
гражданский брак,3762,388,9.35,-1.23,-13.2,Проблемная ситуация.
женат / замужем,11408,931,7.55,0.57,7.5,Хорошая ситуация
в разводе,1110,85,7.11,1.01,14.2,Хорошая ситуация
вдовец / вдова,896,63,6.57,1.55,23.6,Хорошая ситуация


Как видим из таблицы, при применении только одного вида категории, а в данном случае, это по семейному статусу, отклонения от среднего значения, также как и в предыдущем исследовании есть, отклонения от среднего значения в районе 12-13% в проблемную сторону и до 24% в положительную сторону.

Сделаем комбинации с другими категориями и посмотрим меняются ли отклонения, есть ли зависимости в комбинациях. Применим дополнительно по очереди следующие зависимости:
* пол
* рабочий статус
* образование
* уровень доходов
* цель кредита

Категории: "наличие детей" мы уже  использовать с "семейным статусом" не будем, т.к. уже использовали в предыдущем исследовании.

In [54]:
#создаем лист из возможных дополнительных комбинаций
list_addiction_with_family_status = ['gender', 'income_type', 'education', 'purpose_lemma', 'total_income_group']

In [55]:
#применим функцию для  комбинации основной категории с одной дополнительной
create_combination('family_status', list_addiction_with_family_status)

Зависимость категорий family_status и gender:


,debt_group,без просрочек,с просрочкой,avg_debt_%,deviation,deviation_%,category_special_case
family_status,gender,,,,,,
вдовец / вдова,m,44.0,11.0,20.00,-11.88,-59.4,Критический случай!
не женат / не замужем,m,925.0,156.0,14.43,-6.31,-43.7,Особая проблемная ситуация!
гражданский брак,m,1150.0,155.0,11.88,-3.76,-31.6,Проблемная ситуация.
в разводе,m,235.0,24.0,9.27,-1.15,-12.4,Проблемная ситуация.
женат / замужем,m,4178.0,401.0,8.76,-0.64,-7.3,Проблемная ситуация.
гражданский брак,f,2611.0,233.0,8.19,-0.07,-0.9,Проблемная ситуация.
женат / замужем,f,7230.0,530.0,6.83,1.29,18.9,Хорошая ситуация
не женат / не замужем,f,1611.0,118.0,6.82,1.30,19.1,Хорошая ситуация
в разводе,f,875.0,61.0,6.52,1.60,24.5,Хорошая ситуация


Зависимость категорий family_status и income_type:


,debt_group,без просрочек,с просрочкой,avg_debt_%,deviation,deviation_%,category_special_case
family_status,income_type,,,,,,
не женат / не замужем,сотрудник,1314.0,174.0,11.69,-3.57,-30.5,Проблемная ситуация.
гражданский брак,сотрудник,1988.0,239.0,10.73,-2.61,-24.3,Проблемная ситуация.
в разводе,сотрудник,563.0,60.0,9.63,-1.51,-15.7,Проблемная ситуация.
гражданский брак,компаньон,916.0,95.0,9.40,-1.28,-13.6,Проблемная ситуация.
вдовец / вдова,компаньон,88.0,9.0,9.28,-1.16,-12.5,Проблемная ситуация.
не женат / не замужем,компаньон,734.0,72.0,8.93,-0.81,-9.1,Проблемная ситуация.
женат / замужем,сотрудник,5893.0,573.0,8.86,-0.74,-8.4,Проблемная ситуация.
не женат / не замужем,госслужащий,152.0,12.0,7.32,0.80,10.9,Хорошая ситуация
гражданский брак,госслужащий,243.0,18.0,6.90,1.22,17.7,Хорошая ситуация


Зависимость категорий family_status и education:


debt_group                                 без просрочек  с просрочкой  \
family_status         education                                          
гражданский брак      начальное                     45.0          11.0   
вдовец / вдова        неоконченное высшее           11.0           2.0   
в разводе             неоконченное высшее           27.0           4.0   
не женат / не замужем среднее                     1570.0         207.0   
женат / замужем       начальное                    126.0          15.0   
гражданский брак      среднее                     2667.0         308.0   
в разводе             начальное                     18.0           2.0   
не женат / не замужем неоконченное высшее          175.0          18.0   
                      начальное                     31.0           3.0   
женат / замужем       неоконченное высшее          311.0          30.0   
гражданский брак      неоконченное высшее          152.0          14.0   
женат / замужем       среднее                     8082.0         731.0   
в разводе             среднее                      763.0          64.0   
вдовец / вдова        среднее                      725.0          54.0   
гражданский брак      высшее                       898.0          55.0   
не женат / не замужем высшее                       759.0          46.0   
вдовец / вдова        высшее                       129.0           7.0   
женат / замужем       высшее                      2885.0         155.0   
в разводе             высшее                       301.0          15.0   
                      ученая степень                 1.0           NaN   
вдовец / вдова        начальное                     31.0           NaN   
женат / замужем       ученая степень                 4.0           NaN   
не женат / не замужем ученая степень                 1.0           NaN   

debt_group                                 avg_debt_%  deviation  deviation_%  \
family_status         education                                                 
гражданский брак      начальное                 19.64     -11.52        -58.7   
вдовец / вдова        неоконченное высшее       15.38      -7.26        -47.2   
в разводе             неоконченное высшее       12.90      -4.78        -37.1   
не женат / не замужем среднее                   11.65      -3.53        -30.3   
женат / замужем       начальное                 10.64      -2.52        -23.7   
гражданский брак      среднее                   10.35      -2.23        -21.5   
в разводе             начальное                 10.00      -1.88        -18.8   
не женат / не замужем неоконченное высшее        9.33      -1.21        -13.0   
                      начальное                  8.82      -0.70         -7.9   
женат / замужем       неоконченное высшее        8.80      -0.68         -7.7   
гражданский брак      неоконченное высшее        8.43      -0.31         -3.7   
женат / замужем       среднее                    8.29      -0.17         -2.1   
в разводе             среднее                    7.74       0.38          4.9   
вдовец / вдова        среднее                    6.93       1.19         17.2   
гражданский брак      высшее                     5.77       2.35         40.7   
не женат / не замужем высшее                     5.71       2.41         42.2   
вдовец / вдова        высшее                     5.15       2.97         57.7   
женат / замужем       высшее                     5.10       3.02         59.2   
в разводе             высшее                     4.75       3.37         70.9   
                      ученая степень              NaN        NaN          NaN   
вдовец / вдова        начальное                   NaN        NaN          NaN   
женат / замужем       ученая степень              NaN        NaN          NaN   
не женат / не замужем ученая степень              NaN        NaN          NaN   

debt_group                                       category_special_case  
family_status         education                   

Зависимость категорий family_status и purpose_lemma:


debt_group                             без просрочек  с просрочкой  \
family_status         purpose_lemma                                  
гражданский брак      на образование             344            60   
не женат / не замужем на автомобиль              555            82   
гражданский брак      на автомобиль              382            51   
не женат / не замужем на образование             515            62   
гражданский брак      на недвижимость            898            91   
вдовец / вдова        на автомобиль              198            20   
женат / замужем       на автомобиль             2507           229   
                      на образование            2379           216   
не женат / не замужем на недвижимость           1466           130   
гражданский брак      на свадьбу                2138           186   
вдовец / вдова        на образование             184            15   
в разводе             на автомобиль              260            21   
                      на образование             221            17   
                      на недвижимость            629            47   
женат / замужем       на недвижимость           6522           486   
вдовец / вдова        на недвижимость            514            28   

debt_group                             avg_debt_%  deviation  deviation_%  \
family_status         purpose_lemma                                         
гражданский брак      на образование        14.85      -6.73        -45.3   
не женат / не замужем на автомобиль         12.87      -4.75        -36.9   
гражданский брак      на автомобиль         11.78      -3.66        -31.1   
не женат / не замужем на образование        10.75      -2.63        -24.5   
гражданский брак      на недвижимость        9.20      -1.08        -11.7   
вдовец / вдова        на автомобиль          9.17      -1.05        -11.5   
женат / замужем       на автомобиль          8.37      -0.25         -3.0   
                      на образование         8.32      -0.20         -2.4   
не женат / не замужем на недвижимость        8.15      -0.03         -0.4   
гражданский брак      на свадьбу             8.00       0.12          1.5   
вдовец / вдова        на образование         7.54       0.58          7.7   
в разводе             на автомобиль          7.47       0.65          8.7   
                      на образование         7.14       0.98         13.7   
                      на недвижимость        6.95       1.17         16.8   
женат / замужем       на недвижимость        6.93       1.19         17.2   
вдовец / вдова        на недвижимость        5.17       2.95         57.1   

debt_group                                   category_special_case  
family_status         purpose_lemma                                 
гражданский брак      на образование   Особая проблемная ситуация!  
не женат / не замужем на автомобиль    Особая проблемная ситуация!  
гражданский брак      на автомобиль           Проблемная ситуация.  
не женат / не замужем на образование          Проблемная ситуация.  
гражданский брак      на недвижимость         Проблемная ситуация.  
вдовец / вдова        на автомобиль           Проблемная ситуация.  
женат / замужем       на автомобиль           Проблемная ситуация.  
                      на образование          Проблемная ситуация.  
не женат / не замужем на недвижимость         Проблемная ситуация.  
гражданский брак      на свадьбу                  Хорошая ситуация  
вдовец / вдова        на образование              Хорошая ситуация  
в разводе             на автомобиль               Хорошая ситуация  
                      на образование              Хорошая ситуация  
                      на недвижимость             Хорошая ситуация  
женат / замужем       на недвижимость             Хорошая ситуация  
вдовец / вдова        на недвижимость             Хорошая ситуация

Зависимость категорий family_status и total_income_group:


debt_group                                  без просрочек  с просрочкой  \
family_status         total_income_group                                  
не женат / не замужем выше среднего доходы            612            77   
гражданский брак      средние доходы                  948           116   
                      выше среднего доходы            897           106   
не женат / не замужем средние доходы                  675            74   
                      высокие доходы                  640            66   
гражданский брак      низкие доходы                   945            96   
вдовец / вдова        выше среднего доходы            180            17   
не женат / не замужем низкие доходы                   609            57   
в разводе             средние доходы                  272            25   
женат / замужем       средние доходы                 2843           248   
                      низкие доходы                  2809           237   
                      выше среднего доходы           2832           229   
в разводе             низкие доходы                   272            22   
вдовец / вдова        средние доходы                  258            20   
женат / замужем       высокие доходы                 2924           217   
гражданский брак      высокие доходы                  972            70   
вдовец / вдова        высокие доходы                  156            11   
в разводе             выше среднего доходы            278            19   
                      высокие доходы                  288            19   
вдовец / вдова        низкие доходы                   302            15   

debt_group                                  avg_debt_%  deviation  \
family_status         total_income_group                            
не женат / не замужем выше среднего доходы       11.18      -3.06   
гражданский брак      средние доходы             10.90      -2.78   
                      выше среднего доходы       10.57      -2.45   
не женат / не замужем средние доходы              9.88      -1.76   
                      высокие доходы              9.35      -1.23   
гражданский брак      низкие доходы               9.22      -1.10   
вдовец / вдова        выше среднего доходы        8.63      -0.51   
не женат / не замужем низкие доходы               8.56      -0.44   
в разводе             средние доходы              8.42      -0.30   
женат / замужем       средние доходы              8.02       0.10   
                      низкие доходы               7.78       0.34   
                      выше среднего доходы        7.48       0.64   
в разводе             низкие доходы               7.48       0.64   
вдовец / вдова        средние доходы              7.19       0.93   
женат / замужем       высокие доходы              6.91       1.21   
гражданский брак      высокие доходы              6.72       1.40   
вдовец / вдова        высокие доходы              6.59       1.53   
в разводе             выше среднего доходы        6.40       1.72   
                      высокие доходы              6.19       1.93   
вдовец / вдова        низкие доходы               4.73       3.39   

debt_group                                  deviation_% category_special_case  
family_status         total_income_group                                       
не женат / не замужем выше среднего доходы        -27.4  Проблемная ситуация.  
гражданский брак      средние доходы              -25.5  Проблемная ситуация.  
                      выше среднего доходы        -23.2  Проблемная ситуация.  
не женат / не замужем средние доходы              -17.8  Проблемная ситуация.  
                      высокие доходы              -13.2  Проблемная ситуация.  
гражданский брак      низкие доходы               -11.9  Проблемная ситуация.  
вдовец / вдова        выше среднего доходы         -5.9  Проблемная ситуация.  
не женат / не замужем низкие доходы                -5.1  Проблемная ситуация.  
в разводе             средние 

Отметим критические или особые проблемные ситуации есть в комбинациях с "полом", с "образованием", с "целями".

Крайние случаи, как положительные, так и отрицательные есть в комбинациях с "рабочим положением", с "образованием".

In [56]:
#применим функцию для комбинации основной категории с двумя дополнительными
create_combination_twice('family_status', list_addiction_with_family_status)

Зависимость категорий family_status с gender и education:


debt_group                                        без просрочек  с просрочкой  \
family_status         gender education                                          
гражданский брак      m      начальное                     12.0           4.0   
вдовец / вдова        m      среднее                       30.0           9.0   
гражданский брак      f      начальное                     33.0           7.0   
не женат / не замужем m      среднее                      568.0         117.0   
вдовец / вдова        m      высшее                        10.0           2.0   
                      f      неоконченное высшее           11.0           2.0   
в разводе             f      неоконченное высшее           19.0           3.0   
                             начальное                     13.0           2.0   
гражданский брак      m      среднее                      837.0         127.0   
женат / замужем       m      начальное                     64.0           9.0   
в разводе             m      неоконченное высшее            8.0           1.0   
не женат / не замужем m      неоконченное высшее           81.0          10.0   
в разводе             m      среднее                      162.0          19.0   
женат / замужем       m      среднее                     2971.0         322.0   
не женат / не замужем m      высшее                       256.0          27.0   
женат / замужем       m      неоконченное высшее          114.0          12.0   
не женат / не замужем m      начальное                     19.0           2.0   
гражданский брак      f      неоконченное высшее           98.0          10.0   
                             среднее                     1830.0         181.0   
женат / замужем       f      начальное                     62.0           6.0   
                             неоконченное высшее          197.0          18.0   
не женат / не замужем f      среднее                     1002.0          90.0   
                             неоконченное высшее           94.0           8.0   
                             начальное                     12.0           1.0   
гражданский брак      m      высшее                       248.0          20.0   
женат / замужем       f      среднее                     5111.0         409.0   
гражданский брак      m      неоконченное высшее           53.0           4.0   
в разводе             f      среднее                      601.0          45.0   
                      m      высшее                        59.0           4.0   
вдовец / вдова        f      среднее                      695.0          45.0   
женат / замужем       m      высшее                      1027.0          58.0   
гражданский брак      f      высшее                       650.0          35.0   
женат / замужем       f      высшее                      1858.0          97.0   
в разводе             f      высшее                       242.0          11.0   
вдовец / вдова        f      высшее                       119.0           5.0   
не женат / не замужем f      высшее                       503.0          19.0   
в разводе             m      начальное                      5.0           NaN   
                             ученая степень                 1.0           NaN   
вдовец / вдова        f      начальное                     27.0           NaN   
                      m      начальное                      4.0           NaN   
гражданский брак      xna    неоконченное высшее            1.0           NaN   
женат / замужем       f      ученая степень                 2.0           NaN   
                      m      ученая степень                 2.0           NaN   
не женат / не замужем m      ученая степень                 1.0           NaN   

debt_group                                        avg_debt_%  deviation  \
family_status         gender education                                    
гражданский брак      m      начальное                 25.00     -16.88   
вдовец / вдова        m      среднее            

Зависимость категорий family_status с gender и purpose_lemma:


debt_group                                    без просрочек  с просрочкой  \
family_status         gender purpose_lemma                                  
вдовец / вдова        m      на образование             4.0           3.0   
                             на автомобиль              6.0           4.0   
гражданский брак      m      на образование            94.0          23.0   
не женат / не замужем m      на образование           160.0          39.0   
                             на автомобиль            188.0          44.0   
гражданский брак      m      на автомобиль            106.0          16.0   
в разводе             m      на образование            40.0           6.0   
гражданский брак      f      на образование           250.0          37.0   
                      m      на недвижимость          266.0          38.0   
                      f      на автомобиль            276.0          35.0   
не женат / не замужем m      на недвижимость          577.0          73.0   
вдовец / вдова        m      на недвижимость           34.0           4.0   
гражданский брак      m      на свадьбу               684.0          78.0   
женат / замужем       m      на автомобиль            920.0          98.0   
                             на образование           885.0          92.0   
не женат / не замужем f      на автомобиль            367.0          38.0   
в разводе             m      на автомобиль             58.0           6.0   
женат / замужем       m      на недвижимость         2373.0         211.0   
в разводе             m      на недвижимость          137.0          12.0   
гражданский брак      f      на недвижимость          631.0          53.0   
вдовец / вдова        f      на автомобиль            192.0          16.0   
женат / замужем       f      на образование          1494.0         124.0   
                             на автомобиль           1587.0         131.0   
в разводе             f      на автомобиль            202.0          15.0   
гражданский брак      f      на свадьбу              1454.0         108.0   
в разводе             f      на недвижимость          492.0          35.0   
вдовец / вдова        f      на образование           180.0          12.0   
женат / замужем       f      на недвижимость         4149.0         275.0   
не женат / не замужем f      на образование           355.0          23.0   
                             на недвижимость          889.0          57.0   
в разводе             f      на образование           181.0          11.0   
вдовец / вдова        f      на недвижимость          480.0          24.0   
гражданский брак      xna    на недвижимость            1.0           NaN   

debt_group                                    avg_debt_%  deviation  \
family_status         gender purpose_lemma                            
вдовец / вдова        m      на образование        42.86     -34.74   
                             на автомобиль         40.00     -31.88   
гражданский брак      m      на образование        19.66     -11.54   
не женат / не замужем m      на образование        19.60     -11.48   
                             на автомобиль         18.97     -10.85   
гражданский брак      m      на автомобиль         13.11      -4.99   
в разводе             m      на образование        13.04      -4.92   
гражданский брак      f      на образование        12.89      -4.77   
                      m      на недвижимость       12.50      -4.38   
                      f      на автомобиль         11.25      -3.13   
не женат / не замужем m      на недвижимость       11.23      -3.11   
вдовец / вдова        m      на недвижимость       10.53      -2.41   
гражданский брак      m      на свадьбу            10.24      -2.12   
женат / замужем       m      на автомобиль          9.63      -1.51   
                             на образование         9.42      -1.30   
не женат / не замужем f      на автомобиль          9.38      -1.26   
в разводе             m   

Зависимость категорий family_status с gender и total_income_group:


...

debt_group                                             без просрочек  \
family_status         income_type education                            
гражданский брак      госслужащий начальное                      1.0   
                      сотрудник   начальное                     23.0   
женат / замужем       компаньон   начальное                     15.0   
вдовец / вдова        сотрудник   неоконченное высшее            5.0   
                      пенсионер   неоконченное высшее            5.0   
...                                                              ...   
не женат / не замужем компаньон   начальное                      3.0   
                      пенсионер   начальное                      9.0   
                                  неоконченное высшее            5.0   
                      сотрудник   ученая степень                 1.0   
                      студент     высшее                         1.0   

debt_group                                             с просрочкой  \
family_status         income_type education                           
гражданский брак      госслужащий начальное                     1.0   
                      сотрудник   начальное                     8.0   
женат / замужем       компаньон   начальное                     3.0   
вдовец / вдова        сотрудник   неоконченное высшее           1.0   
                      пенсионер   неоконченное высшее           1.0   
...                                                             ...   
не женат / не замужем компаньон   начальное                     NaN   
                      пенсионер   начальное                     NaN   
                                  неоконченное высшее           NaN   
                      сотрудник   ученая степень                NaN   
                      студент     высшее                        NaN   

debt_group                                             avg_debt_%  deviation  \
family_status         income_type education                                    
гражданский брак      госслужащий начальное                 50.00     -41.88   
                      сотрудник   начальное                 25.81     -17.69   
женат / замужем       компаньон   начальное                 16.67      -8.55   
вдовец / вдова        сотрудник   неоконченное высшее       16.67      -8.55   
                      пенсионер   неоконченное высшее       16.67      -8.55   
...                                                           ...        ...   
не женат / не замужем компаньон   начальное                   NaN        NaN   
                      пенсионер   начальное                   NaN        NaN   
                                  неоконченное высшее         NaN        NaN   
                      сотрудник   ученая степень              NaN        NaN   
                      студент     высшее                      NaN        NaN   

debt_group                                             deviation_%  \
family_status         income_type education                          
гражданский брак      госслужащий начальное                  -83.8   
                      сотрудник   начальное                  -68.5   
женат / замужем       компаньон   начальное                  -51.3   
вдовец / вдова        сотрудник   неоконченное высшее        -51.3   
                      пенсионер   неоконченное высшее        -51.3   
...                                                            ...   
не женат / не замужем компаньон   начальное                    NaN   
                      пенсионер   начальное                    NaN   
                                  неоконченное высшее          NaN   
                      сотрудник   ученая степень               NaN   
                      студент     высшее                       NaN   

debt_group                                            category_special_case  
family_status         income_type education                                  
гражданский брак      госслужащ

Зависимость категорий family_status с income_type и purpose_lemma:


debt_group                                             без просрочек  \
family_status         income_type     purpose_lemma                    
вдовец / вдова        госслужащий     на автомобиль              6.0   
гражданский брак      госслужащий     на образование            19.0   
                      сотрудник       на образование           175.0   
                      госслужащий     на автомобиль             21.0   
                      сотрудник       на автомобиль            206.0   
...                                                              ...   
                      предприниматель на свадьбу                 1.0   
женат / замужем       безработный     на недвижимость            NaN   
                      в декрете       на автомобиль              NaN   
                      предприниматель на недвижимость            1.0   
не женат / не замужем студент         на недвижимость            1.0   

debt_group                                             с просрочкой  \
family_status         income_type     purpose_lemma                   
вдовец / вдова        госслужащий     на автомобиль             2.0   
гражданский брак      госслужащий     на образование            4.0   
                      сотрудник       на образование           35.0   
                      госслужащий     на автомобиль             4.0   
                      сотрудник       на автомобиль            39.0   
...                                                             ...   
                      предприниматель на свадьбу                NaN   
женат / замужем       безработный     на недвижимость           1.0   
                      в декрете       на автомобиль             1.0   
                      предприниматель на недвижимость           NaN   
не женат / не замужем студент         на недвижимость           NaN   

debt_group                                             avg_debt_%  deviation  \
family_status         income_type     purpose_lemma                            
вдовец / вдова        госслужащий     на автомобиль         25.00     -16.88   
гражданский брак      госслужащий     на образование        17.39      -9.27   
                      сотрудник       на образование        16.67      -8.55   
                      госслужащий     на автомобиль         16.00      -7.88   
                      сотрудник       на автомобиль         15.92      -7.80   
...                                                           ...        ...   
                      предприниматель на свадьбу              NaN        NaN   
женат / замужем       безработный     на недвижимость         NaN        NaN   
                      в декрете       на автомобиль           NaN        NaN   
                      предприниматель на недвижимость         NaN        NaN   
не женат / не замужем студент         на недвижимость         NaN        NaN   

debt_group                                             deviation_%  \
family_status         income_type     purpose_lemma                  
вдовец / вдова        госслужащий     на автомобиль          -67.5   
гражданский брак      госслужащий     на образование         -53.3   
                      сотрудник       на образование         -51.3   
                      госслужащий     на автомобиль          -49.2   
                      сотрудник       на автомобиль          -49.0   
...                                                            ...   
                      предприниматель на свадьбу               NaN   
женат / замужем       безработный     на недвижимость          NaN   
                      в декрете       на автомобиль            NaN   
                      предприниматель на недвижимость          NaN   
не женат / не замужем студент         на недвижимость          NaN   

debt_group                                                   category_special_case  
family_status         income_type     purpose_lemma                                 
вдовец / вдова   

Зависимость категорий family_status с income_type и total_income_group:


debt_group                                                  без просрочек  \
family_status         income_type     total_income_group                    
женат / замужем       безработный     низкие доходы                     0   
                      в декрете       низкие доходы                     0   
не женат / не замужем сотрудник       выше среднего доходы            278   
вдовец / вдова        компаньон       средние доходы                   20   
не женат / не замужем сотрудник       высокие доходы                  277   
...                                                                   ...   
                      предприниматель выше среднего доходы              0   
                                      высокие доходы                    0   
                      студент         средние доходы                    0   
                                      выше среднего доходы              0   
                                      высокие доходы                    0   

debt_group                                                  с просрочкой  \
family_status         income_type     total_income_group                   
женат / замужем       безработный     низкие доходы                    1   
                      в декрете       низкие доходы                    1   
не женат / не замужем сотрудник       выше среднего доходы            42   
вдовец / вдова        компаньон       средние доходы                   3   
не женат / не замужем сотрудник       высокие доходы                  39   
...                                                                  ...   
                      предприниматель выше среднего доходы             0   
                                      высокие доходы                   0   
                      студент         средние доходы                   0   
                                      выше среднего доходы             0   
                                      высокие доходы                   0   

debt_group                                                  avg_debt_%  \
family_status         income_type     total_income_group                 
женат / замужем       безработный     низкие доходы             100.00   
                      в декрете       низкие доходы             100.00   
не женат / не замужем сотрудник       выше среднего доходы       13.12   
вдовец / вдова        компаньон       средние доходы             13.04   
не женат / не замужем сотрудник       высокие доходы             12.34   
...                                                                ...   
                      предприниматель выше среднего доходы         NaN   
                                      высокие доходы               NaN   
                      студент         средние доходы               NaN   
                                      выше среднего доходы         NaN   
                                      высокие доходы               NaN   

debt_group                                                  deviation  \
family_status         income_type     total_income_group                
женат / замужем       безработный     низкие доходы            -91.88   
                      в декрете       низкие доходы            -91.88   
не женат / не замужем сотрудник       выше среднего доходы      -5.00   
вдовец / вдова        компаньон       средние доходы            -4.92   
не женат / не замужем сотрудник       высокие доходы            -4.22   
...                                                               ...   
                      предприниматель выше среднего доходы        NaN   
                                      высокие доходы              NaN   
                      студент         средние доходы              NaN   
                                      выше среднего доходы        NaN   
                                      высокие доходы              NaN   

debt_group                                                  deviation_%  \
family_status        

**Вывод**

Есть ли зависимость между семейным положением и возвратом кредита в срок?, можно сказать следующее - есть отклонение/зависимость в худшую сторону у заемщиков "не женатых / не замужем" и в "гражданском браке" на 16% и 12,5% соответственно от исходной средней величины. Положительные результаты у категорий "женат/замужем", "в разводе", "вдовец/вдова" на 6,9%, 15,9% и 24,3%.

Особо стоит отменить при комбинации с категорией по "полу", в основном проблемные зоны только у мужского полу, у женского хорошая ситуация.

Похожая ситуакция наблюдается при комбинациях с "полом" и "целями" явное положительное превосходство у женского пола.

Отметим также и зависимость при комбинации с "образованием", также как и с наличием детей, есть связь от уровня образования.

#### Зависимость с учетом уровня доходов

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [57]:
#сформируем сводную таблицу
data_pivot_income = pd.pivot_table(df, index = ['total_income_group'], \
                            columns = 'debt_group', values = 'debt', aggfunc = 'count')

#добавим столбец относительной величины просрочек к общим кредитам
data_pivot_income['avg_debt_%'] = round(data_pivot_income['с просрочкой'] \
                          / (data_pivot_income['с просрочкой'] + data_pivot_income['без просрочек']) * 100, 2)

#добавим столбец, в котором покажем с отклонениями от среднего показателия
data_pivot_income['deviation'] = round(avg_debt - data_pivot_income['avg_debt_%'], 2)

#добавим столбец, показывающий на сколько % отклонение от средней величины
data_pivot_income['deviation_%'] = round(data_pivot_income['deviation'] / data_pivot_income['avg_debt_%'] * 100, 1)

#применим функцию категоризации отклонений от среднего показателия
category_special_case(data_pivot_income)

#выведим таблицу на экран и одновременно отсортируем по столбцу отклонений от среднего
display(data_pivot_income.sort_values('deviation', ascending=True))


debt_group,без просрочек,с просрочкой,avg_debt_%,deviation,deviation_%,category_special_case
total_income_group,,,,,,
средние доходы,4996,483,8.82,-0.70,-7.9,Проблемная ситуация.
выше среднего доходы,4799,448,8.54,-0.42,-4.9,Проблемная ситуация.
низкие доходы,4937,427,7.96,0.16,2.0,Хорошая ситуация
высокие доходы,4980,383,7.14,0.98,13.7,Хорошая ситуация


Как видим из таблицы при применении только одного вида категории, а в данном случае это по уровню доходов, отклонения от среднего значения, также как и в предыдущем исследовании есть, отклонения от среднего значения в районе 13-14% в проблемную сторону и до 18% в положительную сторону.

Поэтому сделаем комбинации с другими категориями и посмотрим меняются ли отклонения, есть ли зависимости в комбинациях. Применим дополнительно по очереди следующие зависимости:
* пол
* рабочий статус
* образование
* цель кредита

Категории: "наличие детей", "семейный статус" мы уже  использовать с "уровнем доходов" не будем, т.к. уже использовали в предыдущем исследовании.

In [58]:
#создаем лист из возможных дополнительных комбинаций
list_addiction_with_total_income_group = ['gender', 'income_type', 'education', 'purpose_lemma']

In [59]:
#применим функцию для  комбинации основной категории с одной дополнительной
create_combination('total_income_group', list_addiction_with_total_income_group)

Зависимость категорий total_income_group и gender:


debt_group                   без просрочек  с просрочкой  avg_debt_%  \
total_income_group   gender                                            
низкие доходы        m                1004           138       12.08   
выше среднего доходы m                1753           224       11.33   
средние доходы       m                1509           179       10.60   
высокие доходы       m                2266           206        8.33   
средние доходы       f                3487           304        8.02   
низкие доходы        f                3933           289        6.85   
выше среднего доходы f                3046           224        6.85   
высокие доходы       f                2713           177        6.12   
                     xna                 1             0        0.00   
низкие доходы        xna                 0             0         NaN   
средние доходы       xna                 0             0         NaN   
выше среднего доходы xna                 0             0         NaN   

debt_group                   deviation  deviation_% category_special_case  
total_income_group   gender                                                
низкие доходы        m           -3.96        -32.8  Проблемная ситуация.  
выше среднего доходы m           -3.21        -28.3  Проблемная ситуация.  
средние доходы       m           -2.48        -23.4  Проблемная ситуация.  
высокие доходы       m           -0.21         -2.5  Проблемная ситуация.  
средние доходы       f            0.10          1.2      Хорошая ситуация  
низкие доходы        f            1.27         18.5      Хорошая ситуация  
выше среднего доходы f            1.27         18.5      Хорошая ситуация  
высокие доходы       f            2.00         32.7      Хорошая ситуация  
                     xna          8.12          inf      Хорошая ситуация  
низкие доходы        xna           NaN          NaN      Крайние значения  
средние доходы       xna           NaN          NaN      Крайние значения  
выше среднего доходы xna           NaN          NaN      Крайние значения

Зависимость категорий total_income_group и income_type:


debt_group                            без просрочек  с просрочкой  avg_debt_%  \
total_income_group   income_type                                                
низкие доходы        безработный                  0             1      100.00   
                     в декрете                    0             1      100.00   
выше среднего доходы сотрудник                 2213           257       10.40   
средние доходы       сотрудник                 2943           323        9.89   
низкие доходы        сотрудник                 2541           270        9.61   
высокие доходы       сотрудник                 2326           211        8.32   
низкие доходы        компаньон                  687            62        8.28   
средние доходы       госслужащий                237            21        8.14   
                     компаньон                  766            66        7.93   
выше среднего доходы компаньон                 1527           131        7.90   
низкие доходы        госслужащий                327            24        6.84   
средние доходы       пенсионер                 1050            73        6.50   
высокие доходы       компаньон                 1722           117        6.36   
                     пенсионер                  559            36        6.05   
выше среднего доходы пенсионер                  623            38        5.75   
высокие доходы       госслужащий                370            19        4.88   
выше среднего доходы госслужащий                436            22        4.80   
низкие доходы        пенсионер                 1381            69        4.76   
высокие доходы       предприниматель              2             0        0.00   
низкие доходы        студент                      1             0        0.00   
высокие доходы       безработный                  1             0        0.00   
низкие доходы        предприниматель              0             0         NaN   
средние доходы       безработный                  0             0         NaN   
                     в декрете                    0             0         NaN   
                     предприниматель              0             0         NaN   
                     студент                      0             0         NaN   
выше среднего доходы безработный                  0             0         NaN   
                     в декрете                    0             0         NaN   
                     предприниматель              0             0         NaN   
                     студент                      0             0         NaN   
высокие доходы       в декрете                    0             0         NaN   
                     студент                      0             0         NaN   

debt_group                            deviation  deviation_%  \
total_income_group   income_type                               
низкие доходы        безработный         -91.88        -91.9   
                     в декрете           -91.88        -91.9   
выше среднего доходы сотрудник            -2.28        -21.9   
средние доходы       сотрудник            -1.77        -17.9   
низкие доходы        сотрудник            -1.49        -15.5   
высокие доходы       сотрудник            -0.20         -2.4   
низкие доходы        компаньон            -0.16         -1.9   
средние доходы       госслужащий          -0.02         -0.2   
                     компаньон             0.19          2.4   
выше среднего доходы компаньон             0.22          2.8   
низкие доходы        госслужащий           1.28         18.7   
средние доходы       пенсионер             1.62         24.9   
высокие доходы       компаньон             1.76         27.7   
                     пенсионер             2.07         34.2   
выше среднего доходы пенсионер             2.37         41.2   
высокие доходы       госслужащий           3.24         66.4   
выше среднего доходы госслужащий           3.32         69.2   
низкие доходы        пенсионе

Зависимость категорий total_income_group и education:


debt_group                                без просрочек  с просрочкой  \
total_income_group   education                                          
средние доходы       начальное                       65            14   
высокие доходы       неоконченное высшее            205            29   
выше среднего доходы начальное                       45             6   
                     среднее                       3250           346   
средние доходы       среднее                       3726           395   
высокие доходы       начальное                       38             4   
низкие доходы        среднее                       3976           365   
высокие доходы       среднее                       2855           258   
средние доходы       неоконченное высшее            166            14   
низкие доходы        неоконченное высшее            133            11   
выше среднего доходы неоконченное высшее            172            14   
низкие доходы        начальное                      103             7   
выше среднего доходы высшее                        1332            82   
низкие доходы        высшее                         724            44   
средние доходы       высшее                        1037            60   
высокие доходы       высшее                        1879            92   
средние доходы       ученая степень                   2             0   
низкие доходы        ученая степень                   1             0   
высокие доходы       ученая степень                   3             0   
выше среднего доходы ученая степень                   0             0   

debt_group                                avg_debt_%  deviation  deviation_%  \
total_income_group   education                                                 
средние доходы       начальное                 17.72      -9.60        -54.2   
высокие доходы       неоконченное высшее       12.39      -4.27        -34.5   
выше среднего доходы начальное                 11.76      -3.64        -31.0   
                     среднее                    9.62      -1.50        -15.6   
средние доходы       среднее                    9.59      -1.47        -15.3   
высокие доходы       начальное                  9.52      -1.40        -14.7   
низкие доходы        среднее                    8.41      -0.29         -3.4   
высокие доходы       среднее                    8.29      -0.17         -2.1   
средние доходы       неоконченное высшее        7.78       0.34          4.4   
низкие доходы        неоконченное высшее        7.64       0.48          6.3   
выше среднего доходы неоконченное высшее        7.53       0.59          7.8   
низкие доходы        начальное                  6.36       1.76         27.7   
выше среднего доходы высшее                     5.80       2.32         40.0   
низкие доходы        высшее                     5.73       2.39         41.7   
средние доходы       высшее                     5.47       2.65         48.4   
высокие доходы       высшее                     4.67       3.45         73.9   
средние доходы       ученая степень             0.00       8.12          inf   
низкие доходы        ученая степень             0.00       8.12          inf   
высокие доходы       ученая степень             0.00       8.12          inf   
выше среднего доходы ученая степень              NaN        NaN          NaN   

debt_group                                      category_special_case  
total_income_group   education                                         
средние доходы       начальное                    Критический случай!  
высокие доходы       неоконченное высшее  Особая проблемная ситуация!  
выше среднего доходы начальное                   Проблемная ситуация.  
                     среднее                     Проблемная ситуация.  
средние доходы       среднее                     Проблемная ситуация.  
высокие доходы       начальное                   Проблемная ситуация.  
низкие доходы        среднее                     Проблем

Зависимость категорий total_income_group и purpose_lemma:


debt_group                            без просрочек  с просрочкой  avg_debt_%  \
total_income_group   purpose_lemma                                              
средние доходы       на образование             914           110       10.74   
выше среднего доходы на автомобиль              913           105       10.31   
низкие доходы        на автомобиль              943           101        9.67   
средние доходы       на свадьбу                 532            56        9.52   
выше среднего доходы на образование             909            94        9.37   
средние доходы       на автомобиль             1051           104        9.00   
низкие доходы        на свадьбу                 548            54        8.97   
                     на образование             951            91        8.73   
высокие доходы       на автомобиль              995            93        8.55   
выше среднего доходы на свадьбу                 511            47        8.42   
высокие доходы       на образование             869            75        7.94   
средние доходы       на недвижимость           2499           213        7.85   
выше среднего доходы на недвижимость           2466           202        7.57   
низкие доходы        на недвижимость           2495           181        6.76   
высокие доходы       на недвижимость           2569           186        6.75   
                     на свадьбу                 547            29        5.03   

debt_group                            deviation  deviation_%  \
total_income_group   purpose_lemma                             
средние доходы       на образование       -2.62        -24.4   
выше среднего доходы на автомобиль        -2.19        -21.2   
низкие доходы        на автомобиль        -1.55        -16.0   
средние доходы       на свадьбу           -1.40        -14.7   
выше среднего доходы на образование       -1.25        -13.3   
средние доходы       на автомобиль        -0.88         -9.8   
низкие доходы        на свадьбу           -0.85         -9.5   
                     на образование       -0.61         -7.0   
высокие доходы       на автомобиль        -0.43         -5.0   
выше среднего доходы на свадьбу           -0.30         -3.6   
высокие доходы       на образование        0.18          2.3   
средние доходы       на недвижимость       0.27          3.4   
выше среднего доходы на недвижимость       0.55          7.3   
низкие доходы        на недвижимость       1.36         20.1   
высокие доходы       на недвижимость       1.37         20.3   
                     на свадьбу            3.09         61.4   

debt_group                           category_special_case  
total_income_group   purpose_lemma                          
средние доходы       на образование   Проблемная ситуация.  
выше среднего доходы на автомобиль    Проблемная ситуация.  
низкие доходы        на автомобиль    Проблемная ситуация.  
средние доходы       на свадьбу       Проблемная ситуация.  
выше среднего доходы на образование   Проблемная ситуация.  
средние доходы       на автомобиль    Проблемная ситуация.  
низкие доходы        на свадьбу       Проблемная ситуация.  
                     на образование   Проблемная ситуация.  
высокие доходы       на автомобиль    Проблемная ситуация.  
выше среднего доходы на свадьбу       Проблемная ситуация.  
высокие доходы       на образование       Хорошая ситуация  
средние доходы       на недвижимость      Хорошая ситуация  
выше среднего доходы на недвижимость      Хорошая ситуация  
низкие доходы        на недвижимость      Хорошая ситуация  
высокие доходы       на недвижимость      Хорошая ситуация  
                     на свадьбу           Хорошая ситуация

Стоит отметить, что комбинация "величины доходов" с одной дополнительной зависимостью показывает, что почти нет критических отклонений (кроме одной) по сравнению с другими основными комбинациями. Есть только одно критическое отклонение - при средних доходах.

Крайние положительные и отрицательные ситуации есть в комбинациях с "рабочим положением", с "образованеим".

In [60]:
#применим функцию для комбинации основной категории с двумя дополнительными
create_combination_twice('total_income_group', list_addiction_with_total_income_group)

Зависимость категорий total_income_group с gender и education:


debt_group                                       без просрочек  с просрочкой  \
total_income_group   gender education                                          
средние доходы       m      начальное                       24             6   
                     f      начальное                       41             8   
выше среднего доходы m      начальное                       24             4   
высокие доходы       f      неоконченное высшее             96            14   
низкие доходы        m      среднее                        815           118   
выше среднего доходы m      среднее                       1264           178   
высокие доходы       m      неоконченное высшее            108            15   
                     f      начальное                       15             2   
средние доходы       m      среднее                       1148           152   
низкие доходы        m      высшее                         124            14   
высокие доходы       m      среднее                       1341           146   
низкие доходы        m      неоконченное высшее             31             3   
средние доходы       f      неоконченное высшее            114            11   
выше среднего доходы f      начальное                       21             2   
средние доходы       f      среднее                       2578           243   
выше среднего доходы m      неоконченное высшее             65             6   
низкие доходы        m      начальное                       33             3   
выше среднего доходы m      высшее                         400            36   
высокие доходы       m      начальное                       23             2   
выше среднего доходы f      среднее                       1986           168   
низкие доходы        f      неоконченное высшее            102             8   
                            среднее                       3161           247   
выше среднего доходы f      неоконченное высшее            107             8   
высокие доходы       f      среднее                       1514           112   
средние доходы       m      высшее                         284            18   
                            неоконченное высшее             52             3   
низкие доходы        f      начальное                       70             4   
средние доходы       f      высшее                         753            42   
высокие доходы       m      высшее                         792            43   
низкие доходы        f      высшее                         600            30   
выше среднего доходы f      высшее                         932            46   
высокие доходы       f      высшее                        1087            49   
средние доходы       m      ученая степень                   1             0   
высокие доходы       m      ученая степень                   2             0   
средние доходы       f      ученая степень                   1             0   
низкие доходы        m      ученая степень                   1             0   
высокие доходы       f      ученая степень                   1             0   
                     xna    неоконченное высшее              1             0   
низкие доходы        f      ученая степень                   0             0   
                     xna    высшее                           0             0   
                            начальное                        0             0   
                            неоконченное высшее              0             0   
                            среднее                          0             0   
                            ученая степень                   0             0   
средние доходы       xna    высшее                           0             0   
                            начальное                        0             0   
                            неоконченное высшее              0             0   
                            среднее                          0             0   


Зависимость категорий total_income_group с gender и purpose_lemma:


debt_group                                   без просрочек  с просрочкой  \
total_income_group   gender purpose_lemma                                  
низкие доходы        m      на автомобиль              172            34   
                            на свадьбу                 112            20   
выше среднего доходы m      на образование             314            52   
                            на автомобиль              338            53   
средние доходы       m      на свадьбу                 145            22   
                            на образование             285            43   
низкие доходы        m      на образование             194            26   
выше среднего доходы m      на свадьбу                 181            21   
низкие доходы        m      на недвижимость            526            58   
высокие доходы       m      на автомобиль              459            50   
средние доходы       m      на недвижимость            770            83   
высокие доходы       m      на образование             390            42   
выше среднего доходы m      на недвижимость            920            98   
средние доходы       f      на образование             629            67   
                     m      на автомобиль              309            31   
                     f      на автомобиль              742            73   
выше среднего доходы f      на автомобиль              575            52   
средние доходы       f      на свадьбу                 387            34   
низкие доходы        f      на автомобиль              771            67   
                            на образование             757            65   
высокие доходы       m      на недвижимость           1171            99   
                     f      на автомобиль              536            43   
выше среднего доходы f      на свадьбу                 330            26   
низкие доходы        f      на свадьбу                 436            34   
средние доходы       f      на недвижимость           1729           130   
выше среднего доходы f      на образование             595            42   
высокие доходы       f      на образование             479            33   
выше среднего доходы f      на недвижимость           1546           104   
низкие доходы        f      на недвижимость           1969           123   
высокие доходы       f      на недвижимость           1397            87   
                     m      на свадьбу                 246            15   
                     f      на свадьбу                 301            14   
                     xna    на недвижимость              1             0   
низкие доходы        xna    на автомобиль                0             0   
                            на недвижимость              0             0   
                            на образование               0             0   
                            на свадьбу                   0             0   
средние доходы       xna    на автомобиль                0             0   
                            на недвижимость              0             0   
                            на образование               0             0   
                            на свадьбу                   0             0   
выше среднего доходы xna    на автомобиль                0             0   
                            на недвижимость              0             0   
                            на образование               0             0   
                            на свадьбу                   0             0   
высокие доходы       xna    на автомобиль                0             0   
                            на образование               0             0   
                            на свадьбу                   0             0   

debt_group                                   avg_debt_%  deviation  \
total_income_group   gender purpose_lemma                            
низкие доходы        m      на автомобиль         16.50    

Зависимость категорий total_income_group с income_type и education:


debt_group                                            без просрочек  \
total_income_group   income_type education                            
низкие доходы        безработный среднее                          0   
                     в декрете   среднее                          0   
                     госслужащий начальное                        2   
средние доходы       сотрудник   начальное                       30   
выше среднего доходы компаньон   начальное                        5   
...                                                             ...   
высокие доходы       студент     высшее                           0   
                                 начальное                        0   
                                 неоконченное высшее              0   
                                 среднее                          0   
                                 ученая степень                   0   

debt_group                                            с просрочкой  \
total_income_group   income_type education                           
низкие доходы        безработный среднее                         1   
                     в декрете   среднее                         1   
                     госслужащий начальное                       1   
средние доходы       сотрудник   начальное                      11   
выше среднего доходы компаньон   начальное                       1   
...                                                            ...   
высокие доходы       студент     высшее                          0   
                                 начальное                       0   
                                 неоконченное высшее             0   
                                 среднее                         0   
                                 ученая степень                  0   

debt_group                                            avg_debt_%  deviation  \
total_income_group   income_type education                                    
низкие доходы        безработный среднее                  100.00     -91.88   
                     в декрете   среднее                  100.00     -91.88   
                     госслужащий начальное                 33.33     -25.21   
средние доходы       сотрудник   начальное                 26.83     -18.71   
выше среднего доходы компаньон   начальное                 16.67      -8.55   
...                                                          ...        ...   
высокие доходы       студент     высшее                      NaN        NaN   
                                 начальное                   NaN        NaN   
                                 неоконченное высшее         NaN        NaN   
                                 среднее                     NaN        NaN   
                                 ученая степень              NaN        NaN   

debt_group                                            deviation_%  \
total_income_group   income_type education                          
низкие доходы        безработный среднее                    -91.9   
                     в декрете   среднее                    -91.9   
                     госслужащий начальное                  -75.6   
средние доходы       сотрудник   начальное                  -69.7   
выше среднего доходы компаньон   начальное                  -51.3   
...                                                           ...   
высокие доходы       студент     высшее                       NaN   
                                 начальное                    NaN   
                                 неоконченное высшее          NaN   
                                 среднее                      NaN   
                                 ученая степень               NaN   

debt_group                                           category_special_case  
total_income_group   income_type education                                  
низкие доходы        безработный среднее               Критический случай!  
        

Зависимость категорий total_income_group с income_type и purpose_lemma:


debt_group                                            без просрочек  \
total_income_group   income_type     purpose_lemma                    
низкие доходы        безработный     на недвижимость              0   
                     в декрете       на автомобиль                0   
                     компаньон       на свадьбу                  70   
выше среднего доходы сотрудник       на автомобиль              395   
                     компаньон       на свадьбу                 151   
...                                                             ...   
высокие доходы       предприниматель на образование               0   
                     студент         на автомобиль                0   
                                     на недвижимость              0   
                                     на образование               0   
                                     на свадьбу                   0   

debt_group                                            с просрочкой  \
total_income_group   income_type     purpose_lemma                   
низкие доходы        безработный     на недвижимость             1   
                     в декрете       на автомобиль               1   
                     компаньон       на свадьбу                 12   
выше среднего доходы сотрудник       на автомобиль              65   
                     компаньон       на свадьбу                 22   
...                                                            ...   
высокие доходы       предприниматель на образование              0   
                     студент         на автомобиль               0   
                                     на недвижимость             0   
                                     на образование              0   
                                     на свадьбу                  0   

debt_group                                            avg_debt_%  deviation  \
total_income_group   income_type     purpose_lemma                            
низкие доходы        безработный     на недвижимость      100.00     -91.88   
                     в декрете       на автомобиль        100.00     -91.88   
                     компаньон       на свадьбу            14.63      -6.51   
выше среднего доходы сотрудник       на автомобиль         14.13      -6.01   
                     компаньон       на свадьбу            12.72      -4.60   
...                                                          ...        ...   
высокие доходы       предприниматель на образование          NaN        NaN   
                     студент         на автомобиль           NaN        NaN   
                                     на недвижимость         NaN        NaN   
                                     на образование          NaN        NaN   
                                     на свадьбу              NaN        NaN   

debt_group                                            deviation_%  \
total_income_group   income_type     purpose_lemma                  
низкие доходы        безработный     на недвижимость        -91.9   
                     в декрете       на автомобиль          -91.9   
                     компаньон       на свадьбу             -44.5   
выше среднего доходы сотрудник       на автомобиль          -42.5   
                     компаньон       на свадьбу             -36.2   
...                                                           ...   
высокие доходы       предприниматель на образование           NaN   
                     студент         на автомобиль            NaN   
                                     на недвижимость          NaN   
                                     на образование           NaN   
                                     на свадьбу               NaN   

debt_group                                                  category_special_case  
total_income_group   income_type     purpose_lemma                                 
низкие доходы        безработный     на недвижимость          Критическ

**Вывод**

На вопрос "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?", можно сказать следующее - есть отклонение/зависимость в худшую сторону у заемщиков со "средними доходами" и с "выше среднего доходами" на 13,8% и 4,6% соответственно от исходной средней величины. Положительные результаты у категорий "низкие доходы", "высокие доходы" на 6%, 18,2% соответственно.

Стоит отметить в комбинациях с полом и целями, есть ярко выраженные положительный перевес женского пола.

Как и в других, ранее проведенных комбинациях, есть явная зависимость от уровня образования.

#### Зависимость с учетом целей кредита

- Как разные цели кредита влияют на его возврат в срок?

In [61]:
#сформируем сводную таблицу
data_pivot_purpose = pd.pivot_table(df, index = ['purpose_lemma'], \
                            columns = 'debt_group', values = 'debt', aggfunc = 'count')

#добавим столбец относительной величины просрочек к общим кредитам
data_pivot_purpose['avg_debt_%'] = round(data_pivot_purpose['с просрочкой'] \
                          / (data_pivot_purpose['с просрочкой'] + data_pivot_purpose['без просрочек']) * 100, 2)

#добавим столбец, в котором покажем с отклонениями от среднего показателия
data_pivot_purpose['deviation'] = round(avg_debt - data_pivot_purpose['avg_debt_%'], 2)

#добавим столбец, показывающий на сколько % отклонение от средней величины
data_pivot_purpose['deviation_%'] = round(data_pivot_purpose['deviation'] / data_pivot_purpose['avg_debt_%'] * 100, 1)

#применим функцию категоризации отклонений от среднего показателия
category_special_case(data_pivot_purpose)

#выведим таблицу на экран и одновременно отсортируем по столбцу отклонений от среднего
display(data_pivot_purpose.sort_values('deviation', ascending=True))


debt_group,без просрочек,с просрочкой,avg_debt_%,deviation,deviation_%,category_special_case
purpose_lemma,,,,,,
на автомобиль,3902,403,9.36,-1.24,-13.2,Проблемная ситуация.
на образование,3643,370,9.22,-1.10,-11.9,Проблемная ситуация.
на свадьбу,2138,186,8.00,0.12,1.5,Хорошая ситуация
на недвижимость,10029,782,7.23,0.89,12.3,Хорошая ситуация


Как видим из таблицы при применении только одного вида категории, а в данном случае это по целям кредита, отклонения от среднего значения, также как и в предыдущем исследовании есть, отклонения от среднего значения в районе 11-13% в проблемную сторону и до 12% в положительную сторону.

Поэтому сделаем комбинации с другими категориями и посмотрим меняются ли отклонения, есть ли зависимости в комбинациях. Применим дополнительно по очереди следующие зависимости:
* пол
* рабочий статус
* образование

Категории: "наличие детей", "семейный статус", "уровень доходов" мы уже  использовать с "целями кредита" не будем, т.к. уже использовали в предыдущем исследовании.

In [62]:
#создаем лист из возможных дополнительных комбинаций
list_addiction_with_purpose_lemma = ['gender', 'income_type', 'education']

In [63]:
#применим функцию для  комбинации основной категории с одной дополнительной
create_combination('purpose_lemma', list_addiction_with_purpose_lemma)

Зависимость категорий purpose_lemma и gender:


debt_group              без просрочек  с просрочкой  avg_debt_%  deviation  \
purpose_lemma   gender                                                       
на образование  m              1183.0         163.0       12.11      -3.99   
на автомобиль   m              1278.0         168.0       11.62      -3.50   
на свадьбу      m               684.0          78.0       10.24      -2.12   
на недвижимость m              3387.0         338.0        9.07      -0.95   
на автомобиль   f              2624.0         235.0        8.22      -0.10   
на образование  f              2460.0         207.0        7.76       0.36   
на свадьбу      f              1454.0         108.0        6.91       1.21   
на недвижимость f              6641.0         444.0        6.27       1.85   
                xna               1.0           NaN         NaN        NaN   

debt_group              deviation_% category_special_case  
purpose_lemma   gender                                     
на образование  m             -32.9  Проблемная ситуация.  
на автомобиль   m             -30.1  Проблемная ситуация.  
на свадьбу      m             -20.7  Проблемная ситуация.  
на недвижимость m             -10.5  Проблемная ситуация.  
на автомобиль   f              -1.2  Проблемная ситуация.  
на образование  f               4.6      Хорошая ситуация  
на свадьбу      f              17.5      Хорошая ситуация  
на недвижимость f              29.5      Хорошая ситуация  
                xna             NaN      Крайние значения

Зависимость категорий purpose_lemma и income_type:


debt_group                       без просрочек  с просрочкой  avg_debt_%  \
purpose_lemma   income_type                                                
на недвижимость безработный                1.0           1.0       50.00   
на автомобиль   сотрудник               1928.0         244.0       11.23   
на образование  сотрудник               1851.0         229.0       11.01   
на свадьбу      компаньон                473.0          52.0        9.90   
на недвижимость сотрудник               5129.0         486.0        8.66   
на свадьбу      сотрудник               1115.0         102.0        8.38   
на образование  госслужащий              237.0          21.0        8.14   
на автомобиль   компаньон                967.0          85.0        8.08   
                госслужащий              263.0          22.0        7.72   
на образование  компаньон                882.0          72.0        7.55   
                пенсионер                673.0          48.0        6.66   
на недвижимость компаньон               2380.0         167.0        6.56   
на автомобиль   пенсионер                744.0          51.0        6.42   
на свадьбу      пенсионер                397.0          25.0        5.92   
на недвижимость пенсионер               1799.0          92.0        4.87   
                госслужащий              718.0          36.0        4.77   
на свадьбу      госслужащий              152.0           7.0        4.40   
на автомобиль   в декрете                  NaN           1.0         NaN   
на недвижимость предприниматель            1.0           NaN         NaN   
                студент                    1.0           NaN         NaN   
на свадьбу      предприниматель            1.0           NaN         NaN   

debt_group                       deviation  deviation_% category_special_case  
purpose_lemma   income_type                                                    
на недвижимость безработный         -41.88        -83.8   Критический случай!  
на автомобиль   сотрудник            -3.11        -27.7  Проблемная ситуация.  
на образование  сотрудник            -2.89        -26.2  Проблемная ситуация.  
на свадьбу      компаньон            -1.78        -18.0  Проблемная ситуация.  
на недвижимость сотрудник            -0.54         -6.2  Проблемная ситуация.  
на свадьбу      сотрудник            -0.26         -3.1  Проблемная ситуация.  
на образование  госслужащий          -0.02         -0.2  Проблемная ситуация.  
на автомобиль   компаньон             0.04          0.5      Хорошая ситуация  
                госслужащий           0.40          5.2      Хорошая ситуация  
на образование  компаньон             0.57          7.5      Хорошая ситуация  
                пенсионер             1.46         21.9      Хорошая ситуация  
на недвижимость компаньон             1.56         23.8      Хорошая ситуация  
на автомобиль   пенсионер             1.70         26.5      Хорошая ситуация  
на свадьбу      пенсионер             2.20         37.2      Хорошая ситуация  
на недвижимость пенсионер             3.25         66.7      Хорошая ситуация  
                госслужащий           3.35         70.2      Хорошая ситуация  
на свадьбу      госслужащий           3.72         84.5      Хорошая ситуация  
на автомобиль   в декрете              NaN          NaN      Крайние значения  
на недвижимость предприниматель        NaN          NaN      Крайние значения  
                студент                NaN          NaN      Крайние значения  
на свадьбу      предприниматель        NaN          NaN      Крайние значения

Зависимость категорий purpose_lemma и education:


debt_group                           без просрочек  с просрочкой  avg_debt_%  \
purpose_lemma   education                                                      
на свадьбу      начальное                     24.0           4.0       14.29   
на недвижимость начальное                    128.0          16.0       11.11   
на автомобиль   начальное                     49.0           6.0       10.91   
                неоконченное высшее          137.0          16.0       10.46   
на образование  среднее                     2594.0         298.0       10.30   
на автомобиль   среднее                     2747.0         314.0       10.26   
на свадьбу      среднее                     1467.0         149.0        9.22   
на образование  неоконченное высшее          119.0          12.0        9.16   
                начальное                     50.0           5.0        9.09   
на недвижимость неоконченное высшее          346.0          34.0        8.95   
                среднее                     6999.0         603.0        7.93   
на свадьбу      неоконченное высшее           74.0           6.0        7.50   
на автомобиль   высшее                       969.0          67.0        6.47   
на образование  высшее                       879.0          55.0        5.89   
на недвижимость высшее                      2551.0         129.0        4.81   
на свадьбу      высшее                       573.0          27.0        4.50   
на недвижимость ученая степень                 5.0           NaN         NaN   
на образование  ученая степень                 1.0           NaN         NaN   

debt_group                           deviation  deviation_%  \
purpose_lemma   education                                     
на свадьбу      начальное                -6.17        -43.2   
на недвижимость начальное                -2.99        -26.9   
на автомобиль   начальное                -2.79        -25.6   
                неоконченное высшее      -2.34        -22.4   
на образование  среднее                  -2.18        -21.2   
на автомобиль   среднее                  -2.14        -20.9   
на свадьбу      среднее                  -1.10        -11.9   
на образование  неоконченное высшее      -1.04        -11.4   
                начальное                -0.97        -10.7   
на недвижимость неоконченное высшее      -0.83         -9.3   
                среднее                   0.19          2.4   
на свадьбу      неоконченное высшее       0.62          8.3   
на автомобиль   высшее                    1.65         25.5   
на образование  высшее                    2.23         37.9   
на недвижимость высшее                    3.31         68.8   
на свадьбу      высшее                    3.62         80.4   
на недвижимость ученая степень             NaN          NaN   
на образование  ученая степень             NaN          NaN   

debt_group                                 category_special_case  
purpose_lemma   education                                         
на свадьбу      начальное            Особая проблемная ситуация!  
на недвижимость начальное                   Проблемная ситуация.  
на автомобиль   начальное                   Проблемная ситуация.  
                неоконченное высшее         Проблемная ситуация.  
на образование  среднее                     Проблемная ситуация.  
на автомобиль   среднее                     Проблемная ситуация.  
на свадьбу      среднее                     Проблемная ситуация.  
на образование  неоконченное высшее         Проблемная ситуация.  
                начальное                   Проблемная ситуация.  
на недвижимость неоконченное высшее         Проблемная ситуация.  
                среднее                         Хорошая ситуация  
на свадьбу      неоконченное высшее             Хорошая ситуация  
на автомобиль   высшее                          Хорошая ситуация  
на образование  высшее                          Хорошая ситуация  
на недвижимость высшее                          Хорошая ситуация  

Особо стоит подчеркнуть зависимость с дополнительной катоегорией по полу, ярко выраженна негативная составляющая по мужскому полу(проблемные ситуации).

Также подчеркнем, ярко прослеживающую зависимость с уровнем образования.

In [64]:
#применим функцию для комбинации основной категории с двумя дополнительными
create_combination_twice('purpose_lemma', list_addiction_with_purpose_lemma)

Зависимость категорий purpose_lemma с gender и education:


,,debt_group,без просрочек,с просрочкой,avg_debt_%,deviation,deviation_%,category_special_case
purpose_lemma,gender,education,,,,,,
на свадьбу,f,начальное,17.0,3.0,15.00,-6.88,-45.9,Особая проблемная ситуация!
на автомобиль,m,начальное,18.0,3.0,14.29,-6.17,-43.2,Особая проблемная ситуация!
на образование,m,среднее,839.0,134.0,13.77,-5.65,-41.0,Особая проблемная ситуация!
на автомобиль,m,среднее,890.0,132.0,12.92,-4.80,-37.2,Особая проблемная ситуация!
на свадьбу,m,начальное,7.0,1.0,12.50,-4.38,-35.0,Особая проблемная ситуация!
на недвижимость,m,начальное,56.0,8.0,12.50,-4.38,-35.0,Особая проблемная ситуация!
на свадьбу,m,среднее,468.0,62.0,11.70,-3.58,-30.6,Проблемная ситуация.
на образование,m,начальное,23.0,3.0,11.54,-3.42,-29.6,Проблемная ситуация.
на автомобиль,f,неоконченное высшее,86.0,11.0,11.34,-3.22,-28.4,Проблемная ситуация.


Зависимость категорий purpose_lemma с income_type и education:


debt_group                                           без просрочек  \
purpose_lemma   income_type     education                            
на образование  госслужащий     начальное                      2.0   
на свадьбу      компаньон       начальное                      3.0   
на недвижимость компаньон       начальное                     11.0   
на автомобиль   сотрудник       начальное                     23.0   
на недвижимость сотрудник       начальное                     63.0   
...                                                            ...   
на образование  сотрудник       ученая степень                 1.0   
на свадьбу      госслужащий     начальное                      1.0   
                                неоконченное высшее            2.0   
                пенсионер       неоконченное высшее            3.0   
                предприниматель высшее                         1.0   

debt_group                                           с просрочкой  avg_debt_%  \
purpose_lemma   income_type     education                                       
на образование  госслужащий     начальное                     1.0       33.33   
на свадьбу      компаньон       начальное                     1.0       25.00   
на недвижимость компаньон       начальное                     3.0       21.43   
на автомобиль   сотрудник       начальное                     5.0       17.86   
на недвижимость сотрудник       начальное                    13.0       17.11   
...                                                           ...         ...   
на образование  сотрудник       ученая степень                NaN         NaN   
на свадьбу      госслужащий     начальное                     NaN         NaN   
                                неоконченное высшее           NaN         NaN   
                пенсионер       неоконченное высшее           NaN         NaN   
                предприниматель высшее                        NaN         NaN   

debt_group                                           deviation  deviation_%  \
purpose_lemma   income_type     education                                     
на образование  госслужащий     начальное               -25.21        -75.6   
на свадьбу      компаньон       начальное               -16.88        -67.5   
на недвижимость компаньон       начальное               -13.31        -62.1   
на автомобиль   сотрудник       начальное                -9.74        -54.5   
на недвижимость сотрудник       начальное                -8.99        -52.5   
...                                                        ...          ...   
на образование  сотрудник       ученая степень             NaN          NaN   
на свадьбу      госслужащий     начальное                  NaN          NaN   
                                неоконченное высшее        NaN          NaN   
                пенсионер       неоконченное высшее        NaN          NaN   
                предприниматель высшее                     NaN          NaN   

debt_group                                          category_special_case  
purpose_lemma   income_type     education                                  
на образование  госслужащий     начальное             Критический случай!  
на свадьбу      компаньон       начальное             Критический случай!  
на недвижимость компаньон       начальное             Критический случай!  
на автомобиль   сотрудник       начальное             Критический случай!  
на недвижимость сотрудник       начальное             Критический случай!  
...                                                                   ...  
на образование  сотрудник       ученая степень           Крайние значения  
на свадьбу      госслужащий     начальное                Крайние значения  
                                неоконченное высшее      Крайние значения  
                пенсионер       неоконченное высшее      Крайние значения  
                предприниматель высшее                   Крайние значения  

[73 rows x 6 colum

**Вывод**

На вопрос "Как разные цели кредита влияют на его возврат в срок?", можно сказать следующее - есть отклонение/зависимость в худшую сторону у заемщиков с целями на "автомобиль" и на "образование" на 13% и 11,5% соответственно от исходной средней величины. Положительные результаты у категорий с целями на "свадьбу", "недвижимость" на 2,2%, 11,7% соответственно.

Особо подчеркнем зависимость с дополнительной катоегорией по полу, т.к. ярко выраженна негативная составляющая по мужскому полу(проблемные ситуации), у женского пола в основе положительные ситуации.

Также подчеркнем, ярко прослеживающую зависимость с уровнем образования, выше образование лучше ситуации с выплатами.

## Шаг 4. Общий вывод

Мы успешно провели все пункты проектной работы - заказа. Провели предобработку данных. Наглядно вывели все необоходимые для работы данные. 

На каждый вопрос заказчика, подробно ответили. Сделали выводы.

Что можно сказать по общей ситуации?

Почти в каждом из 4 вопросов-ситуаций, везде были отклонения категорий в худшую сторону в среднем до 13% от средней величины и в положительную до 24%. На сколько это существенно уже принимать решение заказчикам. Однако разбег от крайностей показателей отклонений получается уже существенный до 40%.

Мужчины, чаще женщин находятся в так называемых зонах - "Критические ситуации" и "Особые проблемные ситуации". Это вполне вероятно из-за того, что мужчинам присуща смелость и, в тоже время чаще, излишняя самоуверенность в своих силах.

Особо стоит подчеркнуть, во всех 4 пунктах исследований, при комбинациях есть четкая зависимость по уровню образования, чем выше образование, тем лучше с выплатой займов. Это, скорее всего, связанно с тем, что люди с более высоким образование, больше просчитывают свои возможности и силы, а также более ответственнее.

При выводе сводных таблиц с применение одной категории, чаще отклонения в худшую сторону составляет до 13% от средней величины почти по всем 4м исследованиям. И только при применении в комбинации с другими категориями, мы видим значительный резонанс зависимостей. 

Зависимость явно проявляется при комбинациях с другими категориями, как с одной доп. категорией, так и особенно с двумя категориями.

Заказчикам стоит также обратить внимание, что многие критические и особые проблемные состояния сформированы на малых выборках, и не совсем будут отражать объективные данные. Стоит с осторожностью их применять в работе.

На больших данных, мы могли бы применить значительно больше комбинаций и результаты были бы намного объективнее. 
